# W&B Results Analysis

Pull experiment metrics from Weights & Biases, build comparison tables, and export LaTeX for the paper.

In [1]:
import os
from pathlib import Path

import wandb
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
try:
    import seaborn as sns
except ImportError:
    sns = None
from dotenv import load_dotenv
from tqdm.auto import tqdm
import warnings

# Load .env from project root (parent of notebooks/)
load_dotenv(Path(__file__).resolve().parent.parent / ".env" if "__file__" in dir() else Path.cwd().parent / ".env")

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", message="ANTLR runtime and generated code versions disagree")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.width", 200)

api = wandb.Api()

/home/vlad.smirnov/miniconda3/envs/lm-polygraph-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: [wandb.Api()] Loaded credentials for https://api.wandb.ai from /home/vlad.smirnov/.netrc.


In [2]:
# ── Configuration ──────────────────────────────────────────────────────────────
#
# Organized by dataset -> strategy. Each group entry is a dict with:
#   "group_url":  link to the wandb group workspace
#   "runs":       list of {"seed": <int>, "run_url": "<url>"}
#
# Single-group strategies (baseline, self_consistency):
#   use a single dict wrapped in a list: [{"group_url": ..., "runs": [...]}]
#
# Multi-group strategies (offline_bon, beam_search, MUR — one group per scorer/aggregation/window):
#   use a list of dicts: [{"group_url": ..., "runs": [...]}, ...]
#
# Fill in the URLs. Entries with empty group_url or runs are skipped.

# ── MATH 500 ─────────────────────────────────────────────────────────────────

MATH500_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/baseline_qwen25_math_7b_instruct_math500",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/baseline_qwen25_math_7b_instruct_math500/runs/qz0418nv"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/baseline_qwen25_math_7b_instruct_math500/runs/3bqhwvgp"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/baseline_qwen25_math_7b_instruct_math500/runs/bjzqimk4"},
        ],
    },
]

MATH500_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/self_consistency_qwen25_math_7b_math500",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/self_consistency_qwen25_math_7b_math500/runs/ky44b84m"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/self_consistency_qwen25_math_7b_math500/runs/gtia4gii"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/self_consistency_qwen25_math_7b_math500/runs/v87vmndj"},
        ],
    },
]

MATH500_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/offline_bon_qwen25_math_7b_instruct_math500_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/offline_bon_qwen25_math_7b_instruct_math500_multi_scorer/runs/c35z6knc"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/offline_bon_qwen25_math_7b_instruct_math500_multi_scorer/runs/d7jh7cbj"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/offline_bon_qwen25_math_7b_instruct_math500_multi_scorer/runs/cz45vmb2"},
        ],
    },
]

MATH500_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_all_mean/runs/jhbjo8i7?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy/runs/xwne3zcp?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy/runs/i6h64mu9?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy/runs/q4irrx7s?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity/runs/j52otygf?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity/runs/2jxse47c?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity/runs/furt84c3?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob/runs/p81rd7g2?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob/runs/xzlye3bd?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob/runs/1gqrkkbz?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_all_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_all_mean/runs/3vyvg771?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_all_mean/runs/a403svwj?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_all_mean/runs/ym9gurc8?nw=nwuserkarantonis"},
        ],
    },
        # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
        # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MATH500_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = []

MATH500_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_5_mean/runs/a5czzn30?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_mean/runs/ymsvscl0?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_mean/runs/pwzb338g?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_mean/runs/35flk5hs?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_mean/runs/wyx5epvb?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_mean/runs/6nvxalna?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_mean/runs/cr02x91m?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_mean/runs/sgz5baov?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_mean/runs/3aal8hbt?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_mean/runs/invmjf46?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_mean/runs/t9izflf2?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_mean/runs/jgwxu1tf?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_mean/runs/1hib1872?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MATH500_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_5_min/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_prm_window_5_min/runs/xsq5qs98?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
{
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_min/runs/s63imxgn?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_min/runs/lboffbb6?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_entropy_window_5_min/runs/3ty4qabr?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_min/runs/71lpyfud?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_min/runs/jdneto0g?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_perplexity_window_5_min/runs/s704cw0h?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_min/runs/i0k09agh?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_min/runs/xk8esovp?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_sequence_prob_window_5_min/runs/w8p45jwz?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_min/runs/9krm0sgu?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_min/runs/z0sfyacp?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/beam_search_qwen25_math_7b_instruct_math500_pd_gap_window_5_min/runs/rxvhmt25?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MATH500_BEAM_SEARCH_RUNS = [
    # *MATH500_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *MATH500_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *MATH500_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *MATH500_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

MATH500_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_prm",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_prm/runs/vd5vmy7u"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_prm/runs/1a781e39"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_prm/runs/rj4rt3i2"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_entropy/runs/tfztdzjl"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_entropy/runs/ggqllnmy"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_entropy/runs/aw88mzyl"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_perplexity",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_perplexity/runs/r6oumz5s"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_perplexity/runs/4ds5ewag"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_perplexity/runs/5xe2x66l"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_sequence_prob/runs/j9a1j7mx"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_sequence_prob/runs/zcdc7nni"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_sequence_prob/runs/otrelz7z"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_uncertainty_pd/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_uncertainty_pd/runs/alb267ge?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_uncertainty_pd/runs/csps436w?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-math500/groups/adaptive_scaling_qwen25_math_7b_instruct_math500_uncertainty_pd/runs/qdng2lkw?nw=nwuserkarantonis"},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MATH500_RUNS = [
    *MATH500_BASELINE_RUNS,
    *MATH500_SELF_CONSISTENCY_RUNS,
    *MATH500_OFFLINE_BON_RUNS,
    *MATH500_BEAM_SEARCH_RUNS,
    *MATH500_MUR_RUNS,
]

In [3]:
{
    "group_url": "",
    "runs": [
        {"seed": 42, "run_url": ""},
        {"seed": 43, "run_url": ""},
        {"seed": 44, "run_url": ""},
    ],
},

({'group_url': '',
  'runs': [{'seed': 42, 'run_url': ''},
   {'seed': 43, 'run_url': ''},
   {'seed': 44, 'run_url': ''}]},)

In [4]:
# ── OlympiadBench ────────────────────────────────────────────────────────────

OLYMPIAD_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/baseline_qwen25_math_7b_instruct_olympiadbench",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/baseline_qwen25_math_7b_instruct_olympiadbench/runs/jzmy8hfc"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/baseline_qwen25_math_7b_instruct_olympiadbench/runs/xlys9l9j"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/baseline_qwen25_math_7b_instruct_olympiadbench/runs/b72k2bg3"},
        ],
    },
]

OLYMPIAD_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/self_consistency_qwen25_math_7b_instruct_olympiadbench",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/self_consistency_qwen25_math_7b_instruct_olympiadbench/runs/fvfzlj30"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/self_consistency_qwen25_math_7b_instruct_olympiadbench/runs/ig5z868b"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/self_consistency_qwen25_math_7b_instruct_olympiadbench/runs/ivwtmfrj"},
        ],
    },
]

OLYMPIAD_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/offline_bon_qwen25_math_7b_instruct_olympiadbench_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/offline_bon_qwen25_math_7b_instruct_olympiadbench_multi_scorer/runs/scg2r5g6"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/offline_bon_qwen25_math_7b_instruct_olympiadbench_multi_scorer/runs/lrbsr0b6"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/offline_bon_qwen25_math_7b_instruct_olympiadbench_multi_scorer/runs/gzpylxq2"},
        ],
    },
]

OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_all_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_all_mean/runs/gui7679t?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy/runs/c6nirtgs?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy/runs/tvpfucx4?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy/runs/juur2mnv?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/xnf5gmhd?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/ljtva92n?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/npatzl31?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/g42a9xqq?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/o3mip5c4?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/c0jumd2j?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_all_mean/runs/50n5dhow?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_all_mean/runs/kpnzm4md?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_all_mean/runs/h69ak088?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = []

OLYMPIAD_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_5_mean/runs/887ddodl?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_mean/runs/erqi0w5r?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_mean/runs/qc0wn4q6?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_mean/runs/bpgur362?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_mean/runs/gb0mtnvx?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_mean/runs/lgbztevg?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_mean/runs/qod2k0j9?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_mean/runs/jnthrv56?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_mean/runs/pw48cch0?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_mean/runs/jilv4iwk?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_mean/runs/43r2f1wt?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_mean/runs/jq2jj0lf?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_mean/runs/fdzegtse?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

OLYMPIAD_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_prm_window_5_min/runs/zqlktvg3?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_min/runs/ygh7bxal?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_min/runs/6bqzf022?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_entropy_window_5_min/runs/pqo1b5f6?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_min/runs/nhi6skmj?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_min/runs/piujnzhv?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_perplexity_window_5_min/runs/5u73qqas?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_min/runs/egbj74e1?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_min/runs/h6v1qr2v?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_sequence_prob_window_5_min/runs/3auslh8z?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_min/runs/9j9zb1s8?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_min/runs/nrliz7lv?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/beam_search_qwen25_math_7b_instruct_olympiadbench_pd_gap_window_5_min/runs/85lk7n3t?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

OLYMPIAD_BEAM_SEARCH_RUNS = [
    *OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *OLYMPIAD_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *OLYMPIAD_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

OLYMPIAD_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_prm",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_prm/runs/i6z1krhj"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_prm/runs/6wvon0ez"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_prm/runs/tdm1eork"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_entropy/runs/vd94h0mo"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_entropy/runs/wf7xwmeq"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_entropy/runs/mysyvdxk"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_perplexity",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/v94x4z5v"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/j0wpb22k"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_perplexity/runs/ma4yye4m"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/9yrxl5jg"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/ux4lvczy"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_sequence_prob/runs/3i0vyyzt"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_uncertainty_pd/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_uncertainty_pd/runs/xcooiyeg?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_uncertainty_pd/runs/0h96fgd9?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-olympiadbench/groups/adaptive_scaling_qwen25_math_7b_instruct_olympiadbench_uncertainty_pd/runs/sysmud1n?nw=nwuserkarantonis"},
        ],
    },
]

OLYMPIAD_RUNS = [
    *OLYMPIAD_BASELINE_RUNS,
    *OLYMPIAD_SELF_CONSISTENCY_RUNS,
    *OLYMPIAD_OFFLINE_BON_RUNS,
    *OLYMPIAD_BEAM_SEARCH_RUNS,
    *OLYMPIAD_MUR_RUNS,
]

In [5]:
# ── Minerva Math ─────────────────────────────────────────────────────────────

MINERVA_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/baseline_qwen25_math_7b_instruct_minerva_math",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/baseline_qwen25_math_7b_instruct_minerva_math/runs/96zj1bj9"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/baseline_qwen25_math_7b_instruct_minerva_math/runs/uyw4bmip"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/baseline_qwen25_math_7b_instruct_minerva_math/runs/qmm3t3wa"},
        ],
    },
]

MINERVA_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/self_consistency_qwen25_math_7b_minerva_math",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/self_consistency_qwen25_math_7b_minerva_math/runs/pnfkhzub"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/self_consistency_qwen25_math_7b_minerva_math/runs/puabpgai"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/self_consistency_qwen25_math_7b_minerva_math/runs/697xpya5"},
        ],
    },
]

MINERVA_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/offline_bon_qwen25_math_7b_instruct_minerva_math_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/offline_bon_qwen25_math_7b_instruct_minerva_math_multi_scorer/runs/7syo0ks4"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/offline_bon_qwen25_math_7b_instruct_minerva_math_multi_scorer/runs/y6udmktu"},
            # TODO: seed 44
        ],
    },
]

MINERVA_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_all_mean/runs/snoxhz7b?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy/runs/962poto8?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy/runs/6s17gujf?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy/runs/pf024sdv?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity/runs/yt9mipkh?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity/runs/795b6z27?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity/runs/za5yp87g?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/4q8i8qwa?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/retc9zoi?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/1fmcdnsh?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_all_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_all_mean/runs/zl7hftku?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_all_mean/runs/xu155a3m?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_all_mean/runs/e201qmog?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MINERVA_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = []

MINERVA_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_5_mean/runs/j4fguv8x?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_mean/runs/wc2stpgz?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_mean/runs/t2jniehz?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_mean/runs/kcfrods7?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_mean/runs/hf1vxco1?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_mean/runs/6y85bwv6?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_mean/runs/wswknexj?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_mean/runs/b7g1qnn1?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_mean/runs/ry44lrr9?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_mean/runs/yqg5cn6j?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_mean/runs/xmcus55r?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_mean/runs/hvw2hvjr?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_mean/runs/nvehp4ya?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MINERVA_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_prm_window_5_min/runs/vj9qje88?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_min/runs/xbxg75pr?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_min/runs/mu1d5ny2?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_entropy_window_5_min/runs/ku1zd1x8?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_min/runs/gq4xwrgy?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_min/runs/ujdvj8w8?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_perplexity_window_5_min/runs/d7zsrmx3?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_min/runs/eninrzri?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_min/runs/dkvyuo46?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_sequence_prob_window_5_min/runs/b106fzn7?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_min/runs/j2ahm5j6?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_min/runs/b1p9fcl8?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/beam_search_qwen25_math_7b_instruct_minerva_math_pd_gap_window_5_min/runs/5avu312g?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

MINERVA_BEAM_SEARCH_RUNS = [
    *MINERVA_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *MINERVA_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *MINERVA_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *MINERVA_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

MINERVA_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_prm",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_prm/runs/06l86wmp"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_prm/runs/rj3c55xp"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva-math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_prm/runs/mqxinev9"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_entropy/runs/6x67bplt"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_entropy/runs/9byp8if5"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_entropy/runs/baw3autg"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_perplexity",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_perplexity/runs/fdqevqae"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_perplexity/runs/v9d5f19e"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_perplexity/runs/1kotu42g"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/hhvycd8w"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/u9jsfizr"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_sequence_prob/runs/s1mch3ag"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_uncertainty_pd/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_uncertainty_pd/runs/u05a1i4m?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_uncertainty_pd/runs/vg6jcpn8?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-minerva_math/groups/adaptive_scaling_qwen25_math_7b_instruct_minerva_math_uncertainty_pd/runs/i6rpgyy0?nw=nwuserkarantonis"},
        ],
    },
]

MINERVA_RUNS = [
    *MINERVA_BASELINE_RUNS,
    *MINERVA_SELF_CONSISTENCY_RUNS,
    *MINERVA_OFFLINE_BON_RUNS,
    *MINERVA_BEAM_SEARCH_RUNS,
    *MINERVA_MUR_RUNS,
]

In [6]:
# ── Gaokao 2023 EN ──────────────────────────────────────────────────────────

GAOKAO_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/baseline_qwen25_math_7b_instruct_gaokao2023en",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/baseline_qwen25_math_7b_instruct_gaokao2023en/runs/kv4jtvgi"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/baseline_qwen25_math_7b_instruct_gaokao2023en/runs/oj4i6jcg"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/baseline_qwen25_math_7b_instruct_gaokao2023en/runs/9jdwpyk5"},
        ],
    },
]

GAOKAO_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/self_consistency_qwen25_math_7b_gaokao2023en",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/self_consistency_qwen25_math_7b_gaokao2023en/runs/fr4b84ia"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/self_consistency_qwen25_math_7b_gaokao2023en/runs/cbuxlxty"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/self_consistency_qwen25_math_7b_gaokao2023en/runs/66ny6wia"},
        ],
    },
]

GAOKAO_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/offline_bon_qwen25_math_7b_instruct_gaokao2023en_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/offline_bon_qwen25_math_7b_instruct_gaokao2023en_multi_scorer/runs/qijvr95c"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/offline_bon_qwen25_math_7b_instruct_gaokao2023en_multi_scorer/runs/jboxbdly"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/offline_bon_qwen25_math_7b_instruct_gaokao2023en_multi_scorer/runs/nh8fx6pk"},
        ],
    },
]

GAOKAO_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_all_mean/runs/8l7qf6cl?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/kn00hw5t?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/3xobx7ih?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/9pdu6tur?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/3o578w75?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/r4i8by0h?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/e2l89n6w?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/fsd1tf66?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/163131v4?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/hp1hq4ff?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_all_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_all_mean/runs/jk68metq?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_all_mean/runs/cylhbr2h?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_all_mean/runs/ykypchxu?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

GAOKAO_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = []

GAOKAO_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_5_mean/runs/1r6s9ry7?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_mean/runs/r3dru40w?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_mean/runs/jlvs82nj?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_mean/runs/p160naq9?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_mean/runs/9me7ishc?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_mean/runs/o5cr0fzk?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_mean/runs/cwq73748?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_mean/runs/1ou0luos?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_mean/runs/dd3hr4mn?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_mean/runs/5t8lox6l?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_mean/runs/05wxrqgd?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_mean/runs/piuuulyg?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_mean/runs/t8kw05ga?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

GAOKAO_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_prm_window_5_min/runs/jtrfp35y?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_min/runs/75zw5ooq?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_min/runs/ne22jlrg?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_entropy_window_5_min/runs/6h66d1po?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_min/runs/yn24c04k?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_min/runs/lsfbd32c?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_perplexity_window_5_min/runs/9sdcszbf?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_min/runs/csx4i9xa?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_min/runs/c7uiu5rq?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_sequence_prob_window_5_min/runs/s980hhyq?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_min/runs/vu34zpxx?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_min/runs/15es444b?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/beam_search_qwen25_math_7b_instruct_gaokao2023en_pd_gap_window_5_min/runs/td9u11e7?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

GAOKAO_BEAM_SEARCH_RUNS = [
    *GAOKAO_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *GAOKAO_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *GAOKAO_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *GAOKAO_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]


GAOKAO_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_prm",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_prm/runs/s0fidl75"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_prm/runs/0ualpblk"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_prm/runs/32rg3g3i"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/hi6vme52"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/8803ke4z"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_entropy/runs/2b58rqn2"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_perplexity",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/nzfniz34?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/t72yerk9"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_perplexity/runs/udabjan5"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/w39yrjlx"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/tg0re0ml"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_sequence_prob/runs/h8ceyznr"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_uncertainty_pd/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_uncertainty_pd/runs/cof8zdfa?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_uncertainty_pd/runs/fvfx85er?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-gaokao2023en/groups/adaptive_scaling_qwen25_math_7b_instruct_gaokao2023en_uncertainty_pd/runs/yx8s5z64?nw=nwuserkarantonis"},
        ],
    },
]

GAOKAO_RUNS = [
    *GAOKAO_BASELINE_RUNS,
    *GAOKAO_SELF_CONSISTENCY_RUNS,
    *GAOKAO_OFFLINE_BON_RUNS,
    *GAOKAO_BEAM_SEARCH_RUNS,
    *GAOKAO_MUR_RUNS,
]

In [7]:
# ── GPQA Diamond ─────────────────────────────────────────────────────────────

GPQA_BASELINE_RUNS = [
    {"group_url": "", "runs": []},
]

GPQA_EXTENDED_THINKING_RUNS = [
    {"group_url": "", "runs": []},
]

GPQA_SELF_CONSISTENCY_RUNS = [
    {"group_url": "", "runs": []},
]

GPQA_OFFLINE_BON_RUNS = []

GPQA_BEAM_SEARCH_RUNS = []

GPQA_RUNS = [
    *GPQA_BASELINE_RUNS,
    *GPQA_EXTENDED_THINKING_RUNS,
    *GPQA_SELF_CONSISTENCY_RUNS,
    *GPQA_OFFLINE_BON_RUNS,
    *GPQA_BEAM_SEARCH_RUNS,
]

In [8]:
# ── AIME 2024 ────────────────────────────────────────────────────────────────

AIME_24_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/xusm290p"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/rb5f45zd"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/ue7upatj"},
            {"seed": 45, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/2pxd5k62"},
            {"seed": 46, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/7wjepb0g"},
            {"seed": 47, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/baseline_qwen3_8b_thinking_aime2024/runs/mmvefyvq"},
        ],
    },
]

AIME_24_EXTENDED_THINKING_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/extended_thinking_qwen3_8b_thinking_aime2024",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/extended_thinking_qwen3_8b_thinking_aime2024/runs/zpdp973a"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/extended_thinking_qwen3_8b_thinking_aime2024/runs/toj7if32"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/extended_thinking_qwen3_8b_thinking_aime2024/runs/nofu424d"},
        ],
    },
]

AIME_24_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/self_consistency_qwen3_8b_thinking_aime2024",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/self_consistency_qwen3_8b_thinking_aime2024/runs/myd2mv7s"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/self_consistency_qwen3_8b_thinking_aime2024/runs/xffzht86"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/self_consistency_qwen3_8b_thinking_aime2024/runs/1pwqvj98"},
        ],
    },
]

AIME_24_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/offline_bon_qwen3_8b_thinking_aime2024_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/offline_bon_qwen3_8b_thinking_aime2024_multi_scorer/runs/sx8686xk?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/offline_bon_qwen3_8b_thinking_aime2024_multi_scorer/runs/yma1zbec?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/offline_bon_qwen3_8b_thinking_aime2024_multi_scorer/runs/jrvn9z5v?nw=nwuserkarantonis"},
        ],
    },
]

{
    "group_url": "",
    "runs": [
        {"seed": 42, "run_url": ""},
        {"seed": 43, "run_url": ""},
        {"seed": 44, "run_url": ""},
    ],
},

AIME_24_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    # TODO
]

AIME_24_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = [
    # TODO
]

AIME_24_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    # TODO
]

AIME_24_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_prm_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_prm_window_5_min/runs/efg8ecwr?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_prm_window_5_min/runs/aqnb3zq1?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_prm_window_5_min/runs/oyhmkas7?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_entropy_window_5_min/runs/0wbteyc1?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_entropy_window_5_min/runs/vchbohcx?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_entropy_window_5_min/runs/qezpea3j?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_perplexity_window_5_min/runs/9n2x29ps?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_perplexity_window_5_min/runs/7xincnfz?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_perplexity_window_5_min/runs/ezyltca8?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_sequence_prob_window_5_min/runs/8vgy2ikx?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_sequence_prob_window_5_min/runs/6d681pee?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_sequence_prob_window_5_min/runs/6b5viy72?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_pd_gap_window_5_min/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_pd_gap_window_5_min/runs/hbdy4fcn?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_pd_gap_window_5_min/runs/4uuttpo2?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/beam_search_qwen3_8b_thinking_aime2024_pd_gap_window_5_min/runs/8tdyfcry?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

AIME_24_BEAM_SEARCH_RUNS = [
    *AIME_24_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *AIME_24_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *AIME_24_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *AIME_24_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

AIME_24_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_prm",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_prm/runs/it11687o"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_prm/runs/jff7wjvc"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_prm/runs/lkq82tbw"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_entropy/runs/0e8v1y8k"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_entropy/runs/0ohmk93u"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_entropy/runs/r3835jcu"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_perplexity/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_perplexity/runs/6akbzucw"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_perplexity/runs/mqw6z61b"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_perplexity/runs/27slpoyj"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_sequence_prob/runs/siootyv4"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_sequence_prob/runs/pbeoa8wj"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2024/groups/adaptive_scaling_qwen3_8b_aime2024_sequence_prob/runs/ehlo2jyt"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime24/groups/adaptive_scaling_qwen3_8b_aime2024_uncertainty_pd/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime24/groups/adaptive_scaling_qwen3_8b_aime2024_uncertainty_pd/runs/ogtic3r3?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime24/groups/adaptive_scaling_qwen3_8b_aime2024_uncertainty_pd/runs/l8o7jyti?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime24/groups/adaptive_scaling_qwen3_8b_aime2024_uncertainty_pd/runs/q81qvm2k?nw=nwuserkarantonis"},
        ],
    },
]

AIME_24_RUNS = [
    *AIME_24_BASELINE_RUNS,
    *AIME_24_EXTENDED_THINKING_RUNS,
    *AIME_24_SELF_CONSISTENCY_RUNS,
    *AIME_24_OFFLINE_BON_RUNS,
    *AIME_24_BEAM_SEARCH_RUNS,
    *AIME_24_MUR_RUNS,
]

In [9]:
# ── AIME 2025 ────────────────────────────────────────────────────────────────

AIME_25_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/5z4xazfd"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/z8961dmg"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/vqkeqd8l"},
            {"seed": 45, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/8ng35cm8"},
            {"seed": 46, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/y6l7gl0x"},
            {"seed": 47, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/baseline_qwen3_8b_thinking_aime2025/runs/bvz1mgw4"},
        ],
    },
]

AIME_25_EXTENDED_THINKING_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/extended_thinking_qwen3_8b_thinking_aime2025",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/extended_thinking_qwen3_8b_thinking_aime2025/runs/ynjskyb9"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/extended_thinking_qwen3_8b_thinking_aime2025/runs/xw5oie56"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/extended_thinking_qwen3_8b_thinking_aime2025/runs/uufgljky"},
        ],
    },
]

AIME_25_SELF_CONSISTENCY_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/self_consistency_qwen3_8b_thinking_aime2025/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/self_consistency_qwen3_8b_thinking_aime2025/runs/e03jj1df"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/self_consistency_qwen3_8b_thinking_aime2025/runs/4o5fp278"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/self_consistency_qwen3_8b_thinking_aime2025/runs/yvpwgde3"},
        ],
    },
]

AIME_25_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/offline_bon_qwen3_8b_thinking_aime2025_multi_scorer",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/offline_bon_qwen3_8b_thinking_aime2025_multi_scorer/runs/1cwevp55"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/offline_bon_qwen3_8b_thinking_aime2025_multi_scorer/runs/ahbar3j1"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/offline_bon_qwen3_8b_thinking_aime2025_multi_scorer/runs/rj0hw4tp"},
        ],
    },
]

{
    "group_url": "",
    "runs": [
        {"seed": 42, "run_url": ""},
        {"seed": 43, "run_url": ""},
        {"seed": 44, "run_url": ""},
    ],
},

AIME_25_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    # TODO
]

AIME_25_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = [
    # TODO
]

AIME_25_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    # TODO
]

AIME_25_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_prm_window_5_min/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_prm_window_5_min/runs/ywizo975?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_prm_window_5_min/runs/czvdxny7?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_prm_window_5_min/runs/o492bbgp?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_entropy_window_5_min/runs/lb0vi3qq?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_entropy_window_5_min/runs/kjwuwim7?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_entropy_window_5_min/runs/utufdnzh?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_perplexity_window_5_min/runs/awws51rg?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_perplexity_window_5_min/runs/qa0rie01?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_perplexity_window_5_min/runs/yhhex0za?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_sequence_prob_window_5_min/runs/mi125juu?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_sequence_prob_window_5_min/runs/7drttyjt?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_sequence_prob_window_5_min/runs/0fsfajui?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_pd_gap_window_5_min/runs/ohpmnuco?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_pd_gap_window_5_min/runs/j5m8fw00?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime2025/groups/beam_search_qwen3_8b_thinking_aime2025_pd_gap_window_5_min/runs/7g1zrrcp?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

AIME_25_BEAM_SEARCH_RUNS = [
    *AIME_25_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *AIME_25_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *AIME_25_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *AIME_25_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

AIME_25_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_prm/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_prm/runs/e42u6g0w"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_prm/runs/3nne68cg"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_prm/runs/upxrnkxo"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_entropy",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_entropy/runs/d8m88x5o"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_entropy/runs/hor6ahou"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_entropy/runs/exlootac"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_perplexity",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_perplexity/runs/3hsaelln"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_perplexity/runs/jertvzag"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_thinking_qwen3_8b_aime2025_perplexity/runs/8o0wi4y3"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_sequence_prob",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_sequence_prob/runs/tce8t6fy"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_sequence_prob/runs/krs5db1u"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_sequence_prob/runs/1ieu2i5k"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_pd_gap/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_pd_gap/runs/8xyihgpg?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_pd_gap/runs/pjd975tp?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-eval-aime25/groups/adaptive_scaling_qwen3_8b_aime2025_pd_gap/runs/8tfhs5vs?nw=nwuserkarantonis"},
        ],
    },
]

AIME_25_RUNS = [
    *AIME_25_BASELINE_RUNS,
    *AIME_25_EXTENDED_THINKING_RUNS,
    *AIME_25_SELF_CONSISTENCY_RUNS,
    *AIME_25_OFFLINE_BON_RUNS,
    *AIME_25_BEAM_SEARCH_RUNS,
    *AIME_25_MUR_RUNS,
]

In [14]:
# ── HumanEval-Plus ────────────────────────────────────────────────────────────

[
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_BASELINE_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/baseline_vllm_qwen3_8b_human_eval_plus/workspace",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/baseline_vllm_qwen3_8b_human_eval_plus/runs/qzbpq9t0?nw=nwuserkarantonis"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/baseline_vllm_qwen3_8b_human_eval_plus/runs/h770ka3u?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/baseline_vllm_qwen3_8b_human_eval_plus/runs/p140hiwl?nw=nwuserkarantonis"},
        ],
    },
]

HUMANEVAL_EXTENDED_THINKING_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/extended_thinking_qwen3_8b_thinking_human_eval_plus/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/extended_thinking_qwen3_8b_thinking_human_eval_plus/runs/lnysauwr?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/extended_thinking_qwen3_8b_thinking_human_eval_plus/runs/rco0ur3e?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/extended_thinking_qwen3_8b_thinking_human_eval_plus/runs/g8zegkd1?nw=nwuserkarantonis"},
        ],
    },
]

HUMANEVAL_SELF_CONSISTENCY_RUNS = []

HUMANEVAL_OFFLINE_BON_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/offline_bon_qwen3_8b_thinking_human_eval_plus_multi_scorer/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/offline_bon_qwen3_8b_thinking_human_eval_plus_multi_scorer/runs/z6vgbfcb?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/offline_bon_qwen3_8b_thinking_human_eval_plus_multi_scorer/runs/gij2g92w?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/offline_bon_qwen3_8b_thinking_human_eval_plus_multi_scorer/runs/hoitoy74?nw=nwuserkarantonis"},
        ],
    },
]

HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS = [
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_all_mean/workspace?nw=nwuserkarantonis",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_all_mean/runs/iidhfsiz?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_all_mean/runs/uvbw2kl5?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_all_mean/runs/e3a0rcup?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_all_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_all_mean/runs/7pzh9edt?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MIN_RUNS = [
    {
        # TODO: add prm
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_all_min/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_all_min/runs/a9s20fwf?nw=nwusertsowehh"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_all_min/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_all_min/runs/48wc71cj?nw=nwusertsowehh"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_all_min/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_all_min/runs/su5e8s4i?nw=nwusertsowehh"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_all_min/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_all_min/runs/xsb1hp9g?nw=nwusertsowehh"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_BEAM_SEARCH_5_STEPS_MEAN_RUNS = [
    {
        # TODO: add prm
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_mean/runs/jwnf1dsb?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_mean/runs/0c7179qy?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_mean/runs/9ig7966n?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_mean/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_mean/runs/r0mics7n?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_BEAM_SEARCH_5_STEPS_MIN_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_prm_window_5_min/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_prm_window_5_min/runs/mwv3ydot?nw=nwusertsowehh"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_prm_window_5_min/runs/diqjyser?nw=nwusertsowehh"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_prm_window_5_min/runs/vuu8s1u9?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_min/runs/wp534gi0?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_min/runs/u69i5npi?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_entropy_window_5_min/runs/niswh0x4?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_min/runs/6p6w68ul?nw=nwuserkarantonis"},
            {"seed": 43, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_min/runs/4g46bojf?nw=nwuserkarantonis"},
            {"seed": 44, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_perplexity_window_5_min/runs/4vo3nvau?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_min/runs/yui20oq0?nw=nwuserkarantonis"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_min/runs/hkqix9z9?nw=nwuserkarantonis"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_sequence_prob_window_5_min/runs/yjnynyg8?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_min/workspace",
        "runs": [
            {"seed": 42, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_min/runs/w72n3060?nw=nwuserkarantonis"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_min/runs/oi9zfd3c?nw=nwuserkarantonis"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/beam_search_qwen3_8b_thinking_human_eval_plus_pd_gap_window_5_min/runs/c4bjiwrf?nw=nwuserkarantonis"},
        ],
    },
    # TODO: LLM_AS_A_CRITIC
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_BEAM_SEARCH_RUNS = [
    *HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
    *HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
    *HUMANEVAL_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
    *HUMANEVAL_BEAM_SEARCH_5_STEPS_MIN_RUNS,
]

HUMANEVAL_MUR_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_prm/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_prm/runs/axsgb0lp?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_prm/runs/sq0dlul0?nw=nwuserkarantonis"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_prm/runs/2218589y?nw=nwuserkarantonis"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_entropy/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_entropy/runs/4g5ugn1v?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_entropy/runs/3d4dfaib?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_entropy/runs/pddw65g8?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_perplexity/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_perplexity/runs/8empwbo4?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_perplexity/runs/t3w67irm?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_perplexity/runs/19mq91wj?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_sequence_prob/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_sequence_prob/runs/hgp1pfn6?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_sequence_prob/runs/uorbvzui?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_sequence_prob/runs/af7aitxx?nw=nwusertsowehh"},
        ],
    },
    # TODO: PD Gap
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_pd_gap/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_pd_gap/runs/2msp2y82?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_pd_gap/runs/05gsk86b?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/adaptive_scaling_vllm_qwen3_8b_human_eval_plus_pd_gap/runs/6kxvjp71?nw=nwusertsowehh"},
        ],
    },
    # TODO: UHEAD
    {
        "group_url": "",
        "runs": [
            {"seed": 42, "run_url": ""},
            {"seed": 43, "run_url": ""},
            {"seed": 44, "run_url": ""},
        ],
    },
]

HUMANEVAL_UNCERT_COT_RUNS = [
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_token/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_token/runs/6fgi9n09?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_token/runs/gnn7ahby?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_token/runs/wqs9v6ae?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_sequence/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_sequence/runs/e7ctq9st?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_sequence/runs/ip36uwc1?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_uncertainty_pd_sequence/runs/vuhbms4x?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "google.com/url?q=https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_token/workspace?nw%3Dnwusertsowehh&sa=D&source=editors&ust=1772036156047322&usg=AOvVaw26Cbh70Rj96_KcWeFRONGm",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_token/runs/010jjrzq?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_token/runs/513w9284?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_token/runs/1kn618b3?nw=nwusertsowehh"},
        ],
    },
    {
        "group_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_sequence/workspace?nw=nwusertsowehh",
        "runs": [
            {"seed": 0, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_sequence/runs/ztolj97l?nw=nwusertsowehh"},
            {"seed": 1, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_sequence/runs/5uwi8jld?nw=nwusertsowehh"},
            {"seed": 2, "run_url": "https://wandb.ai/nlpresearch.group/llm-tts-human-eval-plus/groups/uncert_cot_vllm_qwen3_8b_human_eval_plus_entropy_sequence/runs/8x8numy8?nw=nwusertsowehh"},
        ],
    },
]

HUMANEVAL_RUNS = [
    *HUMANEVAL_BASELINE_RUNS,
    *HUMANEVAL_EXTENDED_THINKING_RUNS,
    *HUMANEVAL_OFFLINE_BON_RUNS,
    *HUMANEVAL_BEAM_SEARCH_RUNS,
    *HUMANEVAL_MUR_RUNS,
    *HUMANEVAL_UNCERT_COT_RUNS,
]


In [15]:
# ── All experiments ──────────────────────────────────────────────────────────

EXPERIMENT_RUNS = [
    *MATH500_RUNS,
    *MINERVA_RUNS,
    *GAOKAO_RUNS,
    *OLYMPIAD_RUNS,
    *GPQA_RUNS,
    *AIME_24_RUNS,
    *AIME_25_RUNS,
    *HUMANEVAL_RUNS,
]

# Which evaluator metric to use as the primary accuracy column
PRIMARY_EVALUATOR = "exact_match"  # or "llm_judge"

GROUP_FILTERS = None  # e.g. ["beam_search", "offline_bon"]

## Seed Coverage Tables

Quick overview of how many seeds (1/2/3) are registered for each (strategy, scorer, dataset) combination.

In [16]:
# ── Seed Coverage: Qwen2.5-Math-7B-Instruct (Non-Thinking) ─────────────────

import re as _re
from urllib.parse import urlparse as _urlparse


def _extract_scorer_from_group_url(url: str) -> str:
    """Extract scorer name from group URL group segment (after /groups/)."""
    path = _urlparse(url).path.strip("/")
    parts = path.split("/")
    # Take the segment right after 'groups/' — ignores trailing /workspace etc.
    try:
        group = parts[parts.index("groups") + 1]
    except (ValueError, IndexError):
        group = parts[-1]
    # Check longest names first to avoid partial matches
    # uncertainty_pd is an alias for pd_gap
    group = group.replace("uncertainty_pd", "pd_gap")
    for scorer in ["sequence_prob", "llm_critic", "multi_scorer",
                   "pd_gap", "prm", "entropy", "perplexity", "uhead"]:
        if group.endswith(f"_{scorer}") or f"_{scorer}_" in group:
            return scorer
    return ""


def _count_seeds_with_url(run_list: list) -> tuple:
    """Count total seeds and return group URL. Returns (count, url)."""
    total = 0
    url = ""
    for entry in run_list:
        if entry.get("group_url"):
            total += sum(1 for r in entry.get("runs", []) if r.get("run_url"))
            if not url:
                url = entry["group_url"]
    return total, url


def _derive_group_url(entry: dict) -> str:
    """Get group_url from entry, falling back to deriving it from a run_url."""
    if entry.get("group_url"):
        return entry["group_url"]
    for run in entry.get("runs", []):
        url = run.get("run_url", "")
        if url and "/groups/" in url:
            # Reconstruct a group URL from the run URL
            # run_url: https://host/entity/project/groups/GROUP/runs/ID
            parts = url.split("/runs/")
            return parts[0]  # everything up to /runs/RUN_ID
    return ""


def _count_seeds_by_scorer_with_url(run_list: list) -> dict:
    """Count seeds per scorer with group URLs. Returns {scorer: (count, url)}."""
    result = {}
    for entry in run_list:
        group_url = _derive_group_url(entry)
        if not group_url:
            continue
        scorer = _extract_scorer_from_group_url(group_url)
        n = sum(1 for r in entry.get("runs", []) if r.get("run_url"))
        result[scorer] = (n, group_url)
    return result


def _color_cell(n_seeds, url=""):
    """Return styled HTML for seed count, optionally as a clickable link."""
    colors = {0: "#e74c3c", 1: "#e67e22", 2: "#f1c40f", 3: "#27ae60"}
    bg = {0: "#fdedec", 1: "#fdebd0", 2: "#fef9e7", 3: "#eafaf1"}
    style = (
        f'background:{bg.get(n_seeds, "#eee")};'
        f'padding:2px 8px;border-radius:3px;'
        f'color:{colors.get(n_seeds, "#333")};font-weight:bold'
    )
    if url:
        return (
            f'<a href="{url}" target="_blank" '
            f'style="{style};text-decoration:none">{n_seeds}</a>'
        )
    return f'<span style="{style}">{n_seeds}</span>'


def build_coverage_table(title, coverage_map, datasets, strategy_order, strategy_labels,
                         multi_scorer_strategies=None, scorer_list=None,
                         scorer_overrides=None):
    """Build an HTML coverage table with rowspan-merged strategy cells.

    Args:
        title: Table heading.
        coverage_map: {dataset: {strategy_key: run_list, ...}, ...}
        datasets: Column order for datasets.
        strategy_order: Row order for strategies.
        strategy_labels: {strategy_key: display_name}
        multi_scorer_strategies: Set of strategy keys that have per-scorer breakdowns.
        scorer_list: Ordered list of scorers for multi-scorer strategies.
    """
    if multi_scorer_strategies is None:
        multi_scorer_strategies = set()
    if scorer_list is None:
        scorer_list = ["prm", "entropy", "perplexity", "sequence_prob", "pd_gap"]
    if scorer_overrides is None:
        scorer_overrides = {}

    # Build row specs: list of (strategy_label, scorer, {ds: (count, url)})
    row_specs = []
    for strat in strategy_order:
        label = strategy_labels.get(strat, strat)
        if strat in multi_scorer_strategies:
            for scorer in scorer_overrides.get(strat, scorer_list):
                cells = {}
                for ds in datasets:
                    by_scorer = _count_seeds_by_scorer_with_url(coverage_map[ds][strat])
                    cells[ds] = by_scorer.get(scorer, (0, ""))
                row_specs.append((label, scorer, cells))
        else:
            cells = {}
            for ds in datasets:
                cells[ds] = _count_seeds_with_url(coverage_map[ds][strat])
            row_specs.append((label, "", cells))

    # Compute rowspans per strategy group
    from collections import Counter
    strategy_counts = Counter(r[0] for r in row_specs)

    th = 'style="padding:6px 12px;border-bottom:2px solid #333;text-align:center"'
    html = f"<h3>{title}</h3>"
    html += '<table style="border-collapse:collapse;font-family:sans-serif;font-size:14px">'
    html += f"<tr><th {th}>Strategy</th><th {th}>Scorer</th>"
    for ds in datasets:
        html += f"<th {th}>{ds}</th>"
    html += "</tr>"

    seen_strategies = set()
    for label, scorer, cells in row_specs:
        first_in_group = label not in seen_strategies
        border = "border-top:1px solid #ccc;" if first_in_group and seen_strategies else ""
        html += f'<tr style="{border}">'

        # Strategy cell: rowspan on first row, skip on subsequent
        if first_in_group:
            rs = strategy_counts[label]
            va = "vertical-align:middle;" if rs > 1 else ""
            html += (
                f'<td rowspan="{rs}" style="padding:4px 12px;font-weight:bold;'
                f'{va}text-align:left">{label}</td>'
            )
            seen_strategies.add(label)

        # Scorer cell (left-aligned)
        scorer_display = scorer if scorer else "—"
        html += f'<td style="padding:4px 12px;text-align:left">{scorer_display}</td>'

        # Dataset cells
        for ds in datasets:
            n, url = cells[ds]
            html += f'<td style="padding:4px 12px;text-align:center">{_color_cell(n, url)}</td>'

        html += "</tr>"

    html += "</table>"
    return html


# ── Build Qwen2.5-Math-7B table ──
BS_SCORER_LIST = ["prm", "entropy", "perplexity", "sequence_prob", "pd_gap", "llm_critic", "uhead"]

QWEN25_COVERAGE = {
    "MATH-500": {
        "baseline": MATH500_BASELINE_RUNS,
        "self_consistency": MATH500_SELF_CONSISTENCY_RUNS,
        "offline_bon": MATH500_OFFLINE_BON_RUNS,
        "bs_all_mean": MATH500_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": MATH500_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": MATH500_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": MATH500_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": MATH500_MUR_RUNS,
    },
    "OlympiadBench": {
        "baseline": OLYMPIAD_BASELINE_RUNS,
        "self_consistency": OLYMPIAD_SELF_CONSISTENCY_RUNS,
        "offline_bon": OLYMPIAD_OFFLINE_BON_RUNS,
        "bs_all_mean": OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": OLYMPIAD_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": OLYMPIAD_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": OLYMPIAD_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": OLYMPIAD_MUR_RUNS,
    },
    "Minerva Math": {
        "baseline": MINERVA_BASELINE_RUNS,
        "self_consistency": MINERVA_SELF_CONSISTENCY_RUNS,
        "offline_bon": MINERVA_OFFLINE_BON_RUNS,
        "bs_all_mean": MINERVA_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": MINERVA_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": MINERVA_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": MINERVA_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": MINERVA_MUR_RUNS,
    },
    "Gaokao 2023 EN": {
        "baseline": GAOKAO_BASELINE_RUNS,
        "self_consistency": GAOKAO_SELF_CONSISTENCY_RUNS,
        "offline_bon": GAOKAO_OFFLINE_BON_RUNS,
        "bs_all_mean": GAOKAO_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": GAOKAO_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": GAOKAO_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": GAOKAO_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": GAOKAO_MUR_RUNS,
    },
}

DATASETS_QWEN25 = ["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"]

html = build_coverage_table(
    title="Qwen2.5-Math-7B-Instruct (Non-Thinking) \u2014 Seed Coverage",
    coverage_map=QWEN25_COVERAGE,
    datasets=DATASETS_QWEN25,
    strategy_order=["baseline", "self_consistency", "offline_bon",
                    "bs_all_mean", "bs_5_mean", "bs_5_min", "mur"],
    strategy_labels={
        "baseline": "Raw CoT<br>(non-thinking)",
        "self_consistency": "Majority Voting<br>(self-consistency)",
        "offline_bon": "Offline BoN",
        "bs_all_mean": "Beam Search<br>(mean, window=all)",
        "bs_5_mean": "Beam Search<br>(mean, window=5)",
        "bs_5_min": "Beam Search<br>(min, window=5)",
        "mur": "MUR",
    },
    multi_scorer_strategies={"mur", "offline_bon", "bs_all_mean", "bs_5_mean", "bs_5_min"},
    scorer_overrides={
        "offline_bon": ["multi_scorer"],
        "bs_all_mean": BS_SCORER_LIST,
        "bs_5_mean": BS_SCORER_LIST,
        "bs_5_min": BS_SCORER_LIST,
    },
)

from IPython.display import HTML
display(HTML(html))

# ── Seed Coverage: Qwen3-8B (Thinking Mode) ──────────────────────────────────

QWEN3_COVERAGE = {
    "AIME-2024": {
        "baseline": AIME_24_BASELINE_RUNS,
        "extended_thinking": AIME_24_EXTENDED_THINKING_RUNS,
        "self_consistency": AIME_24_SELF_CONSISTENCY_RUNS,
        "offline_bon": AIME_24_OFFLINE_BON_RUNS,
        "bs_all_mean": AIME_24_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": AIME_24_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": AIME_24_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": AIME_24_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": AIME_24_MUR_RUNS,
    },
    "AIME-2025": {
        "baseline": AIME_25_BASELINE_RUNS,
        "extended_thinking": AIME_25_EXTENDED_THINKING_RUNS,
        "self_consistency": AIME_25_SELF_CONSISTENCY_RUNS,
        "offline_bon": AIME_25_OFFLINE_BON_RUNS,
        "bs_all_mean": AIME_25_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": AIME_25_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": AIME_25_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": AIME_25_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": AIME_25_MUR_RUNS,
    },
    "HumanEval-Plus": {
        "baseline": HUMANEVAL_BASELINE_RUNS,
        "extended_thinking": HUMANEVAL_EXTENDED_THINKING_RUNS,
        "self_consistency": HUMANEVAL_SELF_CONSISTENCY_RUNS,
        "offline_bon": HUMANEVAL_OFFLINE_BON_RUNS,
        "bs_all_mean": HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MEAN_RUNS,
        "bs_all_min": HUMANEVAL_BEAM_SEARCH_ALL_STEPS_MIN_RUNS,
        "bs_5_mean": HUMANEVAL_BEAM_SEARCH_5_STEPS_MEAN_RUNS,
        "bs_5_min": HUMANEVAL_BEAM_SEARCH_5_STEPS_MIN_RUNS,
        "mur": HUMANEVAL_MUR_RUNS,
    },
}

DATASETS_QWEN3 = ["AIME-2024", "AIME-2025", "HumanEval-Plus"]

html_qwen3 = build_coverage_table(
    title="Qwen3-8B (Thinking Mode) \u2014 Seed Coverage",
    coverage_map=QWEN3_COVERAGE,
    datasets=DATASETS_QWEN3,
    # bs_all_mean, bs_all_min, bs_5_mean temporarily excluded
    strategy_order=["baseline", "extended_thinking", "self_consistency", "offline_bon",
                    "bs_5_min", "mur"],
    strategy_labels={
        "baseline": "Raw CoT<br>(thinking)",
        "extended_thinking": "Extended Thinking CoT",
        "self_consistency": "Majority Voting<br>(self-consistency)",
        "offline_bon": "Offline BoN",
        "bs_5_min": "Beam Search<br>(min, window=5)",
        "mur": "MUR",
    },
    multi_scorer_strategies={"mur", "offline_bon", "bs_5_min"},
    scorer_overrides={
        "offline_bon": ["multi_scorer"],
        "bs_5_min": BS_SCORER_LIST,
    },
)

display(HTML(html_qwen3))


In [23]:
# ── Data Fetching ─────────────────────────────────────────────────────────────
import re
import json as _json
from pathlib import Path as _Path
from urllib.parse import urlparse, parse_qs

# Set to True to force re-fetch from wandb even if cached
FORCE_REFRESH = False
_RUNS_CACHE_DIR = _Path("cache/runs")
_RUNS_CACHE_DIR.mkdir(parents=True, exist_ok=True)


def _unwrap_redirect(url: str) -> str:
    """Unwrap Google redirect URLs to extract the real destination."""
    parsed = urlparse(url)
    host = parsed.hostname or ""
    path = parsed.path or ""
    if "google.com" in host or path.startswith("google.com/"):
        real = parse_qs(parsed.query).get("q", [None])[0]
        if real:
            return real
    return url


def parse_group_url(url: str) -> dict:
    """Extract entity, project, and group name from a wandb group URL."""
    url = _unwrap_redirect(url)
    path = urlparse(url).path.strip("/")
    m = re.match(r"^(?P<entity>[^/]+)/(?P<project>[^/]+)/groups/(?P<group>[^/]+)", path)
    if not m:
        raise ValueError(f"Cannot parse group URL: {url}")
    return m.groupdict()


def parse_run_url(url: str) -> dict:
    """Extract entity, project, and run_id from a wandb run URL.

    Handles both formats:
      .../runs/RUN_ID
      .../groups/GROUP/runs/RUN_ID
    """
    url = _unwrap_redirect(url)
    path = urlparse(url).path.strip("/")
    m = re.match(r"^(?P<entity>[^/]+)/(?P<project>[^/]+)/(?:groups/[^/]+/)?runs/(?P<run_id>[^/]+)", path)
    if not m:
        raise ValueError(f"Cannot parse run URL: {url}")
    return m.groupdict()


def fetch_run(entity: str, project: str, run_id: str, group_name: str, seed: int) -> dict:
    """Fetch a single run and return a flat record dict."""
    run = api.run(f"{entity}/{project}/{run_id}")
    cfg = run.config
    s = run.summary._json_dict

    strategy_cfg = cfg.get("strategy", {})
    scorer_cfg = cfg.get("scorer", {})
    model_cfg = cfg.get("model", {})
    dataset_cfg = cfg.get("dataset", {})

    return {
        # identifiers
        "run_id": run.id,
        "run_name": run.name,
        "group": group_name,
        "state": run.state,
        "project": project,
        "entity": entity,
        "seed": seed,
        # config fields
        "strategy": strategy_cfg.get("type"),
        "scorer": scorer_cfg.get("type"),
        "aggregation": strategy_cfg.get("aggregation"),
        "scoring_window": strategy_cfg.get("scoring_window"),
        "scoring_window_label": strategy_cfg.get("scoring_window_label"),
        "model": model_cfg.get("model_short_name") or model_cfg.get("model_name"),
        "dataset": dataset_cfg.get("data_name"),
        "beam_size": strategy_cfg.get("beam_size"),
        "candidates_per_beam": strategy_cfg.get("candidates_per_beam"),
        "num_paths": strategy_cfg.get("num_paths"),
        "num_candidates": strategy_cfg.get("num_candidates"),
        "max_steps": strategy_cfg.get("max_steps"),
        # summary metrics
        "exact_match": s.get("exact_match/accuracy"),
        "llm_judge_accuracy": next(
            (v for k, v in s.items() if k.startswith("llm_judge") and k.endswith("/accuracy")),
            None,
        ),
        "avg_reasoning_steps": s.get("avg_reasoning_steps_per_trajectory"),
        "total_tokens": s.get("compute/total_tokens"),
        "total_input_tokens": s.get("compute/total_input_tokens"),
        "total_output_tokens": s.get("compute/total_output_tokens"),
        "total_tflops": s.get("compute/total_tflops"),
        "avg_tokens_per_sample": s.get("compute/avg_tokens_per_sample"),
        "avg_output_tokens_per_sample": s.get("compute/avg_output_tokens_per_sample"),
        "avg_tflops_per_sample": s.get("compute/avg_tflops_per_sample"),
        "total_generations": s.get("compute/total_generations"),
        "prm_tflops": s.get("compute/prm_tflops"),
        "total_samples": s.get("total_samples"),
        "completed": s.get("completed"),
        "humaneval_correct": s.get("human_eval_plus/correct"),
        "humaneval_incorrect": s.get("human_eval_plus/incorrect"),
        "humaneval_score": s.get("human_eval_plus/accuracy"),
        # full config & summary for future use
        "_config": cfg,
        "_summary": s,
    }


def _load_cached_run(run_id: str) -> dict | None:
    """Load a cached run record from disk, or return None."""
    cache_file = _RUNS_CACHE_DIR / f"{run_id}.json"
    if cache_file.exists() and not FORCE_REFRESH:
        with open(cache_file) as f:
            return _json.load(f)
    return None


def _save_run_cache(record: dict) -> None:
    """Save a run record to disk cache."""
    cache_file = _RUNS_CACHE_DIR / f"{record['run_id']}.json"
    with open(cache_file, "w") as f:
        _json.dump(record, f, indent=2, default=str)



def _fmt_metrics(rec: dict) -> str:
    """Format metrics for log output — dataset-aware."""
    ds = str(rec.get("dataset", "") or "")
    if "human_eval" in ds:
        hs = rec.get("humaneval_score")
        hs_s = f"{hs:.4f}" if hs is not None else "N/A"
        return f"humaneval_score={hs_s}"
    em = rec.get("exact_match")
    llm = rec.get("llm_judge_accuracy")
    em_s = f"{em:.4f}" if em is not None else "N/A"
    llm_s = f"{llm:.4f}" if llm is not None else "N/A"
    parts = f"exact_match={em_s}  llm_judge={llm_s}"
    if em is not None and llm is not None:
        diff = abs(em - llm)
        if diff > 0.10:
            parts += f"  Δ={diff:.4f} — METRICS DIVERGE >10%"
    return parts


# Validate that all non-empty run_urls are unique across the registry
_all_urls = [
    r["run_url"]
    for entry in EXPERIMENT_RUNS
    for r in entry.get("runs", [])
    if r.get("run_url")
]
_seen, _dupes = set(), set()
for _u in _all_urls:
    (_dupes if _u in _seen else _seen).add(_u)
if _dupes:
    print(f"[warn] {len(_dupes)} duplicate run_url(s) detected:")
    for _u in sorted(_dupes):
        print(f"  {_u}")
else:
    print(f"[ok] all {len(_all_urls)} run URLs are unique")

# Fetch all specified runs (skip entries with empty group_url or runs)
records = []
_cached_count = 0
_fetched_count = 0
for entry in tqdm(EXPERIMENT_RUNS, desc="Fetching runs", leave=False):
    if not entry.get("group_url") or not entry.get("runs"):
        continue

    group_info = parse_group_url(entry["group_url"])
    group_name = group_info["group"]
    tqdm.write(f"Group: {group_name}")

    for run_entry in entry["runs"]:
        seed = run_entry["seed"]
        run_url = run_entry.get("run_url", "")
        if not run_url:
            tqdm.write(f"  [warn] seed={seed} in group '{group_name}' has no run_url, skipping")
            continue
        run_info = parse_run_url(run_url)
        run_id = run_info["run_id"]

        # Try cache first
        cached = _load_cached_run(run_id)
        if cached is not None:
            cached_state = cached.get("state", "")
            # Re-fetch if cached state is not terminal (run might have finished since)
            if cached_state not in ("finished", "crashed", "failed"):
                try:
                    record = fetch_run(run_info["entity"], run_info["project"],
                                       run_id, group_name, seed)
                    _save_run_cache(record)
                    records.append(record)
                    _fetched_count += 1
                    state = record.get("state", "?")
                    tqdm.write(f"  seed={seed}  [{state}]  {_fmt_metrics(record)}  (re-fetched, was {cached_state})")
                    continue
                except Exception as e:
                    tqdm.write(f"  seed={seed}  [re-fetch failed: {e}, using stale cache]")

            # Update group/seed in case registry changed
            cached["group"] = group_name
            cached["seed"] = seed
            records.append(cached)
            _cached_count += 1
            tqdm.write(f"  seed={seed}  [cached, {cached_state}]  {_fmt_metrics(cached)}")
            continue

        try:
            record = fetch_run(run_info["entity"], run_info["project"],
                               run_id, group_name, seed)
            _save_run_cache(record)
            records.append(record)
            _fetched_count += 1
            state = record.get("state", "?")
            tqdm.write(f"  seed={seed}  [{state}]  {_fmt_metrics(record)}")
        except Exception as e:
            tqdm.write(f"  ERROR fetching seed={seed}: {e}")

raw_df = pd.DataFrame(records)
tqdm.write(f"\nTotal runs: {len(raw_df)} ({_cached_count} cached, {_fetched_count} fetched)")

# Warn about runs where "completed" != "total_samples" (dataset size)
# For HumanEval-Plus: check human_eval_plus/correct + human_eval_plus/incorrect == 164
_incomplete = []
_HUMANEVAL_EXPECTED = 164
if not raw_df.empty:
    for _, r in raw_df.iterrows():
        run_link = f"https://wandb.ai/{r['entity']}/{r['project']}/runs/{r['run_id']}"
        dataset = r.get("dataset", "") or ""

        # HumanEval-Plus: use correct + incorrect
        if "human_eval" in dataset:
            hc = r.get("humaneval_correct")
            hi = r.get("humaneval_incorrect")
            if pd.notna(hc) and pd.notna(hi):
                done = int(hc) + int(hi)
                if done != _HUMANEVAL_EXPECTED:
                    _incomplete.append({
                        "run_id": r["run_id"], "group": r["group"], "seed": r["seed"],
                        "state": r.get("state", "?"),
                        "completed": done, "total": _HUMANEVAL_EXPECTED,
                        "pct": done / _HUMANEVAL_EXPECTED * 100,
                        "link": run_link,
                    })
            continue

        # Other datasets: use completed vs total_samples
        comp = r.get("completed")
        total = r.get("total_samples")
        if pd.isna(comp) or pd.isna(total):
            continue
        comp, total = int(comp), int(total)
        if comp != total:
            _incomplete.append({
                "run_id": r["run_id"], "group": r["group"], "seed": r["seed"],
                "state": r.get("state", "?"), "completed": comp, "total": total,
                "pct": comp / total * 100 if total > 0 else 0,
                "link": run_link,
            })

if _incomplete:
    tqdm.write(f"\n⚠ {len(_incomplete)} INCOMPLETE runs detected:")
    for ir in _incomplete:
        tqdm.write(
            f"  {ir['group']}  seed={ir['seed']}  state={ir['state']}  "
            f"completed={ir['completed']}/{ir['total']} ({ir['pct']:.0f}%)  "
            f"{ir['link']}"
        )
else:
    tqdm.write("All runs completed successfully.")

# Report non-finished runs (running, crashed, failed, etc.)
_non_finished = []
if not raw_df.empty:
    for _, r in raw_df.iterrows():
        state = r.get("state", "")
        if state != "finished":
            _non_finished.append({
                "run_id": r["run_id"],
                "group": r["group"],
                "seed": r["seed"],
                "state": state,
                "link": f"https://wandb.ai/{r['entity']}/{r['project']}/runs/{r['run_id']}",
            })

if _non_finished:
    _running = [r for r in _non_finished if r["state"] == "running"]
    _other = [r for r in _non_finished if r["state"] != "running"]
    if _running:
        tqdm.write(f"\n🔄 {len(_running)} RUNNING runs:")
        for r in _running:
            tqdm.write(f"  {r['group']}  seed={r['seed']}  {r['link']}")
    if _other:
        tqdm.write(f"\n❌ {len(_other)} non-finished runs (crashed/failed/other):")
        for r in _other:
            tqdm.write(f"  {r['group']}  seed={r['seed']}  state={r['state']}  {r['link']}")
else:
    tqdm.write("\nAll runs finished.")

# Save cell output to log file
_log_path = _Path("cache/data_fetching.log")
with open(_log_path, "w") as _lf:
    _lf.write(f"Fetched {len(raw_df)} runs ({_cached_count} cached, {_fetched_count} fetched)\n\n")
    for _, r in raw_df.iterrows():
        state = r.get("state", "?")
        _lf.write(f"{r['group']}  seed={r['seed']}  state={state}  "
                   f"{_fmt_metrics(r.to_dict())}\n")
    if _incomplete:
        _lf.write(f"\n{len(_incomplete)} INCOMPLETE runs:\n")
        for ir in _incomplete:
            _lf.write(f"  {ir['group']}  seed={ir['seed']}  "
                       f"completed={ir['completed']}/{ir['total']}  {ir['link']}\n")
    else:
        _lf.write("\nAll runs completed successfully.\n")
    if _non_finished:
        _running_log = [r for r in _non_finished if r["state"] == "running"]
        _other_log = [r for r in _non_finished if r["state"] != "running"]
        if _running_log:
            _lf.write(f"\n{len(_running_log)} RUNNING runs:\n")
            for r in _running_log:
                _lf.write(f"  {r['group']}  seed={r['seed']}  {r['link']}\n")
        if _other_log:
            _lf.write(f"\n{len(_other_log)} non-finished runs (crashed/failed/other):\n")
            for r in _other_log:
                _lf.write(f"  {r['group']}  seed={r['seed']}  state={r['state']}  {r['link']}\n")
    else:
        _lf.write("\nAll runs finished.\n")

print(f"Saved fetch log to {_log_path}")

raw_df.head()


[ok] all 391 run URLs are unique


Fetching runs:   0%|          | 0/184 [00:00<?, ?it/s]

Group: baseline_qwen25_math_7b_instruct_math500
  seed=42  [cached, finished]  exact_match=0.8320  llm_judge=0.8380
  seed=43  [cached, finished]  exact_match=0.8320  llm_judge=0.8380
  seed=44  [cached, finished]  exact_match=0.8320  llm_judge=0.8400
Group: self_consistency_qwen25_math_7b_math500
  seed=42  [cached, finished]  exact_match=0.8620  llm_judge=0.8660
  seed=43  [cached, finished]  exact_match=0.8700  llm_judge=0.8740
  seed=44  [cached, finished]  exact_match=0.8600  llm_judge=0.8660
Group: offline_bon_qwen25_math_7b_instruct_math500_multi_scorer
  seed=42  [cached, finished]  exact_match=0.8500  llm_judge=0.8540
  seed=43  [cached, finished]  exact_match=0.8540  llm_judge=0.8580
  seed=44  [cached, finished]  exact_match=0.8340  llm_judge=0.8420
Group: beam_search_qwen25_math_7b_instruct_math500_prm_window_5_mean
  seed=42  [cached, finished]  exact_match=0.8640  llm_judge=0.8720
  [warn] seed=43 in group 'beam_search_qwen25_math_7b_instruct_math500_prm_window_5_mean' ha

Group: extended_thinking_qwen3_8b_thinking_aime2024


  seed=42  [cached, finished]  exact_match=0.8000  llm_judge=0.8000
  seed=43  [cached, finished]  exact_match=0.8333  llm_judge=0.8333
  seed=44  [cached, finished]  exact_match=0.7333  llm_judge=0.7333
Group: self_consistency_qwen3_8b_thinking_aime2024
  seed=42  [cached, finished]  exact_match=0.8333  llm_judge=0.8333
  seed=43  [cached, finished]  exact_match=0.8333  llm_judge=0.8333
  seed=44  [cached, finished]  exact_match=0.8000  llm_judge=0.8000
Group: offline_bon_qwen3_8b_thinking_aime2024_multi_scorer
  seed=42  [cached, finished]  exact_match=0.7000  llm_judge=0.7000
  seed=43  [cached, finished]  exact_match=0.7333  llm_judge=0.7333
  seed=44  [cached, finished]  exact_match=0.8000  llm_judge=0.8000
Group: beam_search_qwen3_8b_thinking_aime2024_prm_window_5_min
  seed=42  [cached, finished]  exact_match=0.7000  llm_judge=0.7000
  seed=43  [cached, finished]  exact_match=0.7000  llm_judge=0.7000
  seed=44  [cached, finished]  exact_match=0.7333  llm_judge=0.7333
Group: beam

,run_id,run_name,group,state,project,entity,seed,strategy,scorer,aggregation,scoring_window,scoring_window_label,model,dataset,beam_size,candidates_per_beam,num_paths,num_candidates,max_steps,exact_match,llm_judge_accuracy,avg_reasoning_steps,total_tokens,total_input_tokens,total_output_tokens,total_tflops,avg_tokens_per_sample,avg_output_tokens_per_sample,avg_tflops_per_sample,total_generations,prm_tflops,total_samples,completed,humaneval_correct,humaneval_incorrect,_config,_summary,humaneval_score
0,qz0418nv,2026-02-16_baseline_qwen25_math_offi...,baseline_qwen25_math_7b_instruct_mat...,finished,llm-tts-eval-math500,nlpresearch.group,42,baseline,entropy,None,NaN,None,qwen25_math_7b_instruct,math,NaN,NaN,NaN,None,NaN,0.832,0.838,1.0,366654.0,49470.0,317184.0,5133.156,733.308,634.368,10.266312,500.0,NaN,500.0,500.0,NaN,NaN,"{'model': {'type': 'vllm', 'device':...","{'_runtime': 337, '_step': 500, '_ti...",NaN
1,3bqhwvgp,2026-02-16_baseline_qwen25_math_offi...,baseline_qwen25_math_7b_instruct_mat...,finished,llm-tts-eval-math500,nlpresearch.group,43,baseline,entropy,None,NaN,None,qwen25_math_7b_instruct,math,NaN,NaN,NaN,None,NaN,0.832,0.838,1.0,366654.0,49470.0,317184.0,5133.156,733.308,634.368,10.266312,500.0,NaN,500.0,500.0,NaN,NaN,"{'model': {'type': 'vllm', 'device':...","{'_runtime': 335, '_step': 500, '_ti...",NaN
2,bjzqimk4,2026-02-16_baseline_qwen25_math_offi...,baseline_qwen25_math_7b_instruct_mat...,finished,llm-tts-eval-math500,nlpresearch.group,44,baseline,entropy,None,NaN,None,qwen25_math_7b_instruct,math,NaN,NaN,NaN,None,NaN,0.832,0.840,1.0,366654.0,49470.0,317184.0,5133.156,733.308,634.368,10.266312,500.0,NaN,500.0,500.0,NaN,NaN,"{'model': {'type': 'vllm', 'device':...","{'_runtime': 312, '_step': 500, '_ti...",NaN
3,ky44b84m,2026-02-16_self_consistency_vllm_qwe...,self_consistency_qwen25_math_7b_math500,finished,llm-tts-eval-math500,nlpresearch.group,42,self_consistency,entropy,None,NaN,None,qwen25_math_7b_instruct,math,NaN,NaN,8.0,None,NaN,0.862,0.866,1.0,2545296.0,49470.0,2495826.0,35634.144,5090.592,4991.652,71.268288,4000.0,NaN,500.0,500.0,NaN,NaN,"{'model': {'type': 'vllm', 'device':...","{'_runtime': 794, '_step': 500, '_ti...",NaN
4,gtia4gii,2026-02-16_self_consistency_vllm_qwe...,self_consistency_qwen25_math_7b_math500,finished,llm-tts-eval-math500,nlpresearch.group,43,self_consistency,entropy,None,NaN,None,qwen25_math_7b_instruct,math,NaN,NaN,8.0,None,NaN,0.870,0.874,1.0,2554304.0,49470.0,2504834.0,35760.256,5108.608,5009.668,71.520512,4000.0,NaN,500.0,500.0,NaN,NaN,"{'model': {'type': 'vllm', 'device':...","{'_runtime': 859, '_step': 500, '_ti...",NaN


In [49]:
# ── Data Cleaning & Parsing ───────────────────────────────────────────────────

def parse_group_name(group: str | None) -> dict:
    """Best-effort extraction of structured fields from the group name.

    Expected patterns:
      {strategy}_{model}_{dataset}
      {strategy}_{model}_{dataset}_{scorer}
      {strategy}_{model}_{dataset}_{scorer}_{window}_{aggregation}
    """
    result = {"_group_strategy": None, "_group_model": None,
              "_group_dataset": None, "_group_scorer": None,
              "_group_window": None, "_group_aggregation": None}
    if not group:
        return result

    known_strategies = {
        "baseline", "chain_of_thought", "self_consistency",
        "online_bon", "offline_bon", "beam_search",
        "uncertainty_cot", "extended_thinking",
        "adaptive_scaling", "deepconf",
    }
    known_scorers = {
        "prm", "entropy", "perplexity", "sequence_prob",
        "uncertainty", "uncertainty_pd", "uncertainty_uhead",
    }
    known_aggregations = {"mean", "min", "max", "sum", "product", "median"}
    known_datasets = {
        "minerva_math", "math500", "aime2024", "aime2025",
        "gaokao2023en", "human_eval_plus", "olympiadbench",
    }

    parts = group.split("_")

    # Greedy match strategy prefix (try longest first)
    strategy = None
    for length in range(min(3, len(parts)), 0, -1):
        candidate = "_".join(parts[:length])
        if candidate in known_strategies:
            strategy = candidate
            parts = parts[length:]
            break
    result["_group_strategy"] = strategy

    # Scan remaining parts for known tokens
    remaining = "_".join(parts)
    for ds in sorted(known_datasets, key=len, reverse=True):
        if ds in remaining:
            result["_group_dataset"] = ds
            remaining = remaining.replace(ds, "", 1)
            break
    for sc in sorted(known_scorers, key=len, reverse=True):
        if f"_{sc}" in f"_{remaining}":
            result["_group_scorer"] = sc
            remaining = remaining.replace(sc, "", 1)
            break
    for ag in known_aggregations:
        if f"_{ag}" in f"_{remaining}":
            result["_group_aggregation"] = ag
            break
    # window: look for a bare integer or "all"
    for p in remaining.split("_"):
        if p.isdigit():
            result["_group_window"] = p
            break
        if p == "all":
            result["_group_window"] = "all"
            break

    # model: whatever remains after removing known tokens is likely the model
    for tok in [result["_group_dataset"], result["_group_scorer"],
                result["_group_aggregation"], result["_group_window"]]:
        if tok:
            remaining = remaining.replace(tok, "", 1)
    model_str = "_".join(p for p in remaining.split("_") if p)
    result["_group_model"] = model_str or None

    return result


df = raw_df.copy()

# Parse group names to fill missing config columns
parsed = df["group"].apply(parse_group_name).apply(pd.Series)
df = pd.concat([df, parsed], axis=1)

# Fill missing config from parsed group name
for col, gcol in [("strategy", "_group_strategy"), ("scorer", "_group_scorer"),
                   ("aggregation", "_group_aggregation"),
                   ("scoring_window", "_group_window"),
                   ("dataset", "_group_dataset"), ("model", "_group_model")]:
    df[col] = df[col].fillna(df[gcol])

# Drop helper columns
df.drop(columns=[c for c in df.columns if c.startswith("_group_")], inplace=True)

# Filter to finished runs only
n_before = len(df)
df = df[df["state"] == "finished"].copy()
print(f"Kept {len(df)}/{n_before} finished runs")

# Optional group filter
if GROUP_FILTERS:
    mask = df["group"].apply(lambda g: any(f in (g or "") for f in GROUP_FILTERS))
    df = df[mask].copy()
    print(f"After group filter: {len(df)} runs")

# Create human-readable project label from wandb project name
PROJECT_LABEL_MAP = {
    "llm-tts-eval-math500": "MATH-500",
    "llm-tts-eval-minerva-math": "Minerva Math",
    "llm-tts-eval-minerva_math": "Minerva Math",
    "llm-tts-eval-gaokao2023en": "Gaokao 2023 EN",
    "llm-tts-eval-olympiadbench": "OlympiadBench",
    "llm-tts-eval-gpqa-diamond": "GPQA Diamond",
    "llm-tts-eval-gpqa_diamond": "GPQA Diamond",
    "llm-tts-eval-aime24": "AIME 2024",
    "llm-tts-eval-aime2024": "AIME 2024",
    "llm-tts-eval-aime25": "AIME 2025",
    "llm-tts-eval-aime2025": "AIME 2025",
    "llm-tts-human-eval-plus": "HumanEval-Plus",
}
df["project_label"] = df["project"].map(PROJECT_LABEL_MAP).fillna(df["project"])

# Fill humaneval_score for HumanEval-Plus from correct / (correct + incorrect)
# (fallback if human_eval_plus/accuracy was missing from wandb summary)
if "humaneval_correct" in df.columns and "humaneval_incorrect" in df.columns:
    _he_mask = (
        df["humaneval_score"].isna()
        & df["humaneval_correct"].notna()
        & df["humaneval_incorrect"].notna()
    )
    _he_total = df.loc[_he_mask, "humaneval_correct"] + df.loc[_he_mask, "humaneval_incorrect"]
    df.loc[_he_mask, "humaneval_score"] = df.loc[_he_mask, "humaneval_correct"] / _he_total
    _filled = _he_mask.sum()
    if _filled:
        print(f"Filled humaneval_score for {_filled} HumanEval-Plus runs from correct/incorrect")

# ── Hardcoded EM fixes ────────────────────────────────────────────────────────
# AIME 2025 self-consistency: exact_match was computed with buggy extraction code.
# Override with llm_judge_accuracy (verified correct).
_AIME25_SC_BUGGY_RUNS = {"e03jj1df", "4o5fp278", "yvpwgde3"}
_fix_mask = df["run_id"].isin(_AIME25_SC_BUGGY_RUNS)
if _fix_mask.any():
    _before = df.loc[_fix_mask, ["run_id", "exact_match", "llm_judge_accuracy"]].copy()
    df.loc[_fix_mask, "exact_match"] = df.loc[_fix_mask, "llm_judge_accuracy"]
    for _, row in _before.iterrows():
        print(f"  Fixed {row['run_id']}: EM {row['exact_match']:.4f} → {row['llm_judge_accuracy']:.4f} (using LLM judge)")
    print(f"Fixed {_fix_mask.sum()} AIME 2025 self-consistency runs (buggy exact_match → llm_judge)")

# Normalize accuracy to percentage
for col in ["exact_match", "llm_judge_accuracy", "humaneval_score"]:
    if col in df.columns:
        # If values look like fractions (0-1), convert to pct
        mask = df[col].notna() & (df[col] <= 1.0)
        df.loc[mask, col] = df.loc[mask, col] * 100

print(f"\nStrategies: {sorted(df['strategy'].dropna().unique())}")
print(f"Scorers:    {sorted(df['scorer'].dropna().unique())}")
print(f"Datasets:   {sorted(df['project_label'].dropna().unique())}")
df[["strategy", "scorer", "aggregation", "scoring_window", "project_label", "dataset", "model",
    "exact_match", "llm_judge_accuracy", "humaneval_score", "total_tflops"]].sample(20)


Kept 387/391 finished runs
  Fixed e03jj1df: EM 0.2667 → 0.7000 (using LLM judge)
  Fixed 4o5fp278: EM 0.3667 → 0.8000 (using LLM judge)
  Fixed yvpwgde3: EM 0.4000 → 0.7000 (using LLM judge)
Fixed 3 AIME 2025 self-consistency runs (buggy exact_match → llm_judge)

Strategies: ['adaptive', 'baseline', 'beam_search', 'extended_thinking', 'offline_best_of_n', 'self_consistency', 'uncertainty_cot']
Scorers:    ['entropy', 'perplexity', 'prm', 'sequence_prob', 'uncertainty_pd']
Datasets:   ['AIME 2024', 'AIME 2025', 'Gaokao 2023 EN', 'HumanEval-Plus', 'MATH-500', 'Minerva Math', 'OlympiadBench']


,strategy,scorer,aggregation,scoring_window,project_label,dataset,model,exact_match,llm_judge_accuracy,humaneval_score,total_tflops
131,beam_search,uncertainty_pd,mean,all,Gaokao 2023 EN,gaokao2023en,qwen25_math_7b_instruct,72.207792,76.363636,NaN,310784.936
106,adaptive,sequence_prob,None,None,Minerva Math,minerva_math,qwen25_math_7b_instruct,41.544118,47.426471,NaN,25621.666
202,beam_search,perplexity,mean,5.0,OlympiadBench,olympiadbench,qwen25_math_7b_instruct,43.851852,47.259259,NaN,703044.356
3,self_consistency,entropy,None,None,MATH-500,math,qwen25_math_7b_instruct,86.200000,86.600000,NaN,35634.144
183,offline_best_of_n,entropy,None,None,OlympiadBench,olympiadbench,qwen25_math_7b_instruct,41.185185,45.481481,NaN,79853.466
383,uncertainty_cot,uncertainty_pd,None,None,HumanEval-Plus,human_eval_plus,qwen3_8b,NaN,NaN,79.268293,61168.384
362,beam_search,uncertainty_pd,min,5.0,HumanEval-Plus,human_eval_plus,vllm_thinking_qwen3_8b,NaN,NaN,82.926829,1003545.232
162,adaptive,prm,None,None,Gaokao 2023 EN,gaokao2023en,qwen25_math_7b_instruct,70.649351,75.324675,NaN,133779.422
77,beam_search,perplexity,mean,5.0,Minerva Math,minerva_math,qwen25_math_7b_instruct,42.647059,48.529412,NaN,181546.652
83,beam_search,uncertainty_pd,mean,5.0,Minerva Math,minerva_math,qwen25_math_7b_instruct,41.544118,47.058824,NaN,190619.464


In [50]:
# ── Seed Averaging ────────────────────────────────────────────────────────────

CONFIG_COLS = ["strategy", "scorer", "aggregation", "scoring_window",
               "model", "dataset", "project_label",
               "beam_size", "candidates_per_beam", "num_paths", "num_candidates"]

METRIC_COLS = ["exact_match", "llm_judge_accuracy", "humaneval_score", "avg_reasoning_steps",
               "total_tokens", "total_tflops", "avg_tokens_per_sample",
               "avg_output_tokens_per_sample", "avg_tflops_per_sample"]


def aggregate_seeds(df: pd.DataFrame) -> pd.DataFrame:
    """Group by config columns and compute mean/std over seeds."""
    present_cfg = [c for c in CONFIG_COLS if c in df.columns]
    present_met = [c for c in METRIC_COLS if c in df.columns]

    grouped = df.groupby(present_cfg, dropna=False)
    agg = grouped[present_met].agg(["mean", "std", "count"]).reset_index()

    # Flatten multi-level columns
    flat_cols = []
    for col in agg.columns:
        if isinstance(col, tuple) and col[1]:
            flat_cols.append(f"{col[0]}_{col[1]}")
        else:
            flat_cols.append(col[0] if isinstance(col, tuple) else col)
    agg.columns = flat_cols

    # Add a formatted "mean +/- std" column for the primary metric
    for m in present_met:
        mean_col, std_col = f"{m}_mean", f"{m}_std"
        if mean_col in agg.columns:
            agg[f"{m}_fmt"] = agg.apply(
                lambda r: f"{r[mean_col]:.1f} +/- {r[std_col]:.1f}"
                if pd.notna(r[std_col]) and r.get(f"{m}_count", 0) > 1
                else (f"{r[mean_col]:.1f}" if pd.notna(r[mean_col]) else ""),
                axis=1,
            )
    return agg


agg_df = aggregate_seeds(df)
print(f"Aggregated configs: {len(agg_df)}")
agg_df.head()

Aggregated configs: 143


,strategy,scorer,aggregation,scoring_window,model,dataset,project_label,beam_size,candidates_per_beam,num_paths,num_candidates,exact_match_mean,exact_match_std,exact_match_count,llm_judge_accuracy_mean,llm_judge_accuracy_std,llm_judge_accuracy_count,humaneval_score_mean,humaneval_score_std,humaneval_score_count,avg_reasoning_steps_mean,avg_reasoning_steps_std,avg_reasoning_steps_count,total_tokens_mean,total_tokens_std,total_tokens_count,total_tflops_mean,total_tflops_std,total_tflops_count,avg_tokens_per_sample_mean,avg_tokens_per_sample_std,avg_tokens_per_sample_count,avg_output_tokens_per_sample_mean,avg_output_tokens_per_sample_std,avg_output_tokens_per_sample_count,avg_tflops_per_sample_mean,avg_tflops_per_sample_std,avg_tflops_per_sample_count,exact_match_fmt,llm_judge_accuracy_fmt,humaneval_score_fmt,avg_reasoning_steps_fmt,total_tokens_fmt,total_tflops_fmt,avg_tokens_per_sample_fmt,avg_output_tokens_per_sample_fmt,avg_tflops_per_sample_fmt
0,adaptive,entropy,NaN,NaN,qwen25_math_7b_instruct,gaokao2023en,Gaokao 2023 EN,NaN,NaN,NaN,NaN,68.398268,0.912178,3,72.900433,1.227485,3,NaN,NaN,0,6.079654,0.058677,3,2.833733e+06,7.604322e+04,3,39672.257333,1064.605083,3,7360.344589,197.514858,3,4039.057143,183.845981,3,103.044824,2.765208,3,68.4 +/- 0.9,72.9 +/- 1.2,,6.1 +/- 0.1,2833732.7 +/- 76043.2,39672.3 +/- 1064.6,7360.3 +/- 197.5,4039.1 +/- 183.8,103.0 +/- 2.8
1,adaptive,entropy,NaN,NaN,qwen25_math_7b_instruct,math,MATH-500,NaN,NaN,NaN,NaN,84.000000,0.529150,3,84.533333,0.642910,3,NaN,NaN,0,6.054667,0.117377,3,3.527878e+06,1.084599e+05,3,49390.292000,1518.438853,3,7055.756000,216.919836,3,3910.163333,81.139192,3,98.780584,3.036878,3,84.0 +/- 0.5,84.5 +/- 0.6,,6.1 +/- 0.1,3527878.0 +/- 108459.9,49390.3 +/- 1518.4,7055.8 +/- 216.9,3910.2 +/- 81.1,98.8 +/- 3.0
2,adaptive,entropy,NaN,NaN,qwen25_math_7b_instruct,minerva_math,Minerva Math,NaN,NaN,NaN,NaN,39.460784,2.582268,3,45.098039,2.502522,3,NaN,NaN,0,6.133578,0.161695,3,1.726833e+06,7.008064e+04,3,24175.666667,981.128893,3,6348.651961,257.649394,3,2792.047794,80.581655,3,88.881127,3.607092,3,39.5 +/- 2.6,45.1 +/- 2.5,,6.1 +/- 0.2,1726833.3 +/- 70080.6,24175.7 +/- 981.1,6348.7 +/- 257.6,2792.0 +/- 80.6,88.9 +/- 3.6
3,adaptive,entropy,NaN,NaN,qwen25_math_7b_instruct,olympiadbench,OlympiadBench,NaN,NaN,NaN,NaN,40.246914,0.905200,3,43.753086,0.986419,3,NaN,NaN,0,7.374815,0.046069,3,6.541236e+06,1.045451e+05,3,91577.308667,1463.631096,3,9690.720494,154.881597,3,4365.265185,94.415847,3,135.670087,2.168342,3,40.2 +/- 0.9,43.8 +/- 1.0,,7.4 +/- 0.0,6541236.3 +/- 104545.1,91577.3 +/- 1463.6,9690.7 +/- 154.9,4365.3 +/- 94.4,135.7 +/- 2.2
4,adaptive,entropy,NaN,NaN,qwen3_8b,human_eval_plus,HumanEval-Plus,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,88.821138,0.931418,3,69.495935,1.374433,3,4.520602e+07,1.064643e+06,3,723296.293333,17034.285254,3,275646.453252,6491.724563,3,16522.615854,626.947448,3,4410.343252,103.867593,3,,,88.8 +/- 0.9,69.5 +/- 1.4,45206018.3 +/- 1064642.8,723296.3 +/- 17034.3,275646.5 +/- 6491.7,16522.6 +/- 626.9,4410.3 +/- 103.9


In [51]:
# ── Pivot Table Helper ────────────────────────────────────────────────────────

def make_comparison_table(
    df: pd.DataFrame,
    row_field: str,
    col_field: str,
    value_field: str = "exact_match_fmt",
    filter_dict: dict | None = None,
    title: str | None = None,
) -> pd.DataFrame:
    """Build a pivot table from the aggregated DataFrame."""
    sub = df.copy()
    if filter_dict:
        for k, v in filter_dict.items():
            if isinstance(v, list):
                sub = sub[sub[k].isin(v)]
            else:
                sub = sub[sub[k] == v]

    if sub.empty:
        print("No data after filtering.")
        return pd.DataFrame()

    pivot = sub.pivot_table(
        index=row_field,
        columns=col_field,
        values=value_field,
        aggfunc="first",
    )
    if title:
        sep = "=" * len(title)
        print(f"\n{sep}")
        print(title)
        print(sep)
    return pivot


# ── Unified Results Table (Strategy x Scorer → Datasets) ─────────────────────

STRATEGY_LABELS = {
    "baseline": "Baseline",
    "extended_thinking": "Ext. Thinking",
    "self_consistency": "Self-Consistency",
    "offline_best_of_n": "Offline BoN",
    "beam_search": "Beam Search",
    "adaptive": "MUR",
    "online_best_of_n": "Online BoN",
}

# Strategies that have a single scorer row (no per-scorer breakdown)
SINGLE_SCORER_STRATEGIES = {"baseline", "self_consistency", "extended_thinking"}


def build_unified_results_table(
    model_df,
    datasets,
    metrics=None,
    strategy_order=None,
    strategy_labels=None,
):
    """Build a unified table: rows = (Strategy, Scorer), columns = datasets (x metrics).

    Parameters
    ----------
    metrics : str or list of (col_name, display_name) tuples
        Single column name (flat columns) or list of (col, label) for MultiIndex columns.
        Default: "exact_match_fmt" (single metric, flat columns).
    """
    if strategy_order is None:
        strategy_order = [
            "baseline", "extended_thinking", "self_consistency",
            "offline_best_of_n", "beam_search", "adaptive",
        ]
    if strategy_labels is None:
        strategy_labels = STRATEGY_LABELS

    # Normalize metrics spec
    if metrics is None:
        metrics = "exact_match_fmt"
    if isinstance(metrics, str):
        _metric_specs = [(metrics, None)]  # single metric, flat columns
        _multi_col = False
    else:
        _metric_specs = metrics  # list of (col_name, display_name)
        _multi_col = True

    def _extract(match_df):
        """Extract metric values from a single-row match."""
        if match_df.empty:
            return {label or col: "" for col, label in _metric_specs}
        r = match_df.iloc[0]
        return {label or col: r.get(col, "") for col, label in _metric_specs}

    rows = []
    row_index = []

    for strat in strategy_order:
        strat_df = model_df[model_df["strategy"] == strat]
        if strat_df.empty:
            continue

        if strat == "beam_search":
            variants = (
                strat_df[["aggregation", "scoring_window"]]
                .drop_duplicates()
                .sort_values(["scoring_window", "aggregation"])
            )
            for _, vr in variants.iterrows():
                agg, win = vr["aggregation"], vr["scoring_window"]
                win_label = f"window={win}" if pd.notna(win) else "window=all"
                agg_label = agg if pd.notna(agg) else "mean"
                label = f"Beam Search\n({agg_label}, {win_label})"

                sub = strat_df[
                    (strat_df["aggregation"] == agg)
                    & (strat_df["scoring_window"] == win)
                ]
                scorers = sorted(sub["scorer"].dropna().unique())
                for scorer in scorers:
                    row = {}
                    for ds in datasets:
                        vals = _extract(sub[(sub["scorer"] == scorer) & (sub["project_label"] == ds)])
                        if _multi_col:
                            for k, v in vals.items():
                                row[(ds, k)] = v
                        else:
                            row[ds] = list(vals.values())[0]
                    rows.append(row)
                    row_index.append((label, scorer))

        elif strat in SINGLE_SCORER_STRATEGIES:
            label = strategy_labels.get(strat, strat)
            row = {}
            for ds in datasets:
                vals = _extract(strat_df[strat_df["project_label"] == ds])
                if _multi_col:
                    for k, v in vals.items():
                        row[(ds, k)] = v
                else:
                    row[ds] = list(vals.values())[0]
            rows.append(row)
            row_index.append((label, "\u2014"))

        else:
            label = strategy_labels.get(strat, strat)
            scorers = sorted(strat_df["scorer"].dropna().unique())
            for scorer in scorers:
                row = {}
                for ds in datasets:
                    match = strat_df[
                        (strat_df["scorer"] == scorer) & (strat_df["project_label"] == ds)
                    ]
                    vals = _extract(match)
                    if _multi_col:
                        for k, v in vals.items():
                            row[(ds, k)] = v
                    else:
                        row[ds] = list(vals.values())[0]
                rows.append(row)
                row_index.append((label, scorer))

    if not rows:
        print("No data found.")
        return pd.DataFrame()

    mi_rows = pd.MultiIndex.from_tuples(row_index, names=["Strategy", "Scorer"])
    if _multi_col:
        metric_labels = [label or col for col, label in _metric_specs]
        mi_cols = pd.MultiIndex.from_tuples(
            [(ds, m) for ds in datasets for m in metric_labels],
            names=["Dataset", "Metric"],
        )
        return pd.DataFrame(rows, index=mi_rows, columns=mi_cols)
    return pd.DataFrame(rows, index=mi_rows, columns=datasets)



def style_results_table(df):
    """Apply styling: bold + centered strategy names, alternating row colours.
    Uses CSS nth-child selectors for reliable row coloring."""
    _GROUP_COLORS = [("#ffffff", "#f0f0f0"), ("#f0f0f0", "#ffffff")]

    base_styles = [
        {"selector": "th.row_heading.level0",
         "props": [("vertical-align", "middle"), ("font-weight", "bold"),
                    ("text-align", "left"), ("padding", "6px 10px")]},
        {"selector": "th.row_heading.level1",
         "props": [("text-align", "left"), ("padding", "4px 10px")]},
        {"selector": "td",
         "props": [("text-align", "center"), ("padding", "4px 8px")]},
        {"selector": "th.col_heading",
         "props": [("text-align", "center"), ("padding", "4px 8px")]},
        {"selector": "th.index_name",
         "props": [("text-align", "center"), ("padding", "4px 8px")]},
    ]

    if not hasattr(df.index, "get_level_values"):
        return df.style.set_table_styles(base_styles)

    groups = list(df.index.get_level_values(0))
    group_idx = 0
    row_in_group = 0
    for i, g in enumerate(groups):
        if i > 0 and g != groups[i - 1]:
            group_idx = 1 - group_idx
            row_in_group = 0
        palette = _GROUP_COLORS[group_idx]
        color = palette[row_in_group % 2]
        # nth-child is 1-indexed; target both td and th in that row
        base_styles.append({
            "selector": f"tbody tr:nth-child({i + 1})",
            "props": [("background-color", color)],
        })
        row_in_group += 1

    return df.style.set_table_styles(base_styles)

print("make_comparison_table(), build_unified_results_table(), style_results_table() defined")


make_comparison_table(), build_unified_results_table(), style_results_table() defined


In [52]:
# ── Qwen2.5-Math-7B-Instruct (Non-Thinking) ──────────────────────────────────

_QWEN25_DATASETS = ["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"]
_model_df = agg_df[agg_df["model"] == "qwen25_math_7b_instruct"].copy()

# Exclude beam search (mean, window=all) — incomplete runs
_bs_exclude = (
    (_model_df["strategy"] == "beam_search")
    & (_model_df["aggregation"] == "mean")
    & (_model_df["scoring_window"].isna() | _model_df["scoring_window"].astype(str).isin(["all", "nan", "None"]))
)
_model_df = _model_df[~_bs_exclude].copy()
print(f"Qwen2.5-Math-7B-Instruct: {len(_model_df)} aggregated configs")

tbl = build_unified_results_table(
    _model_df,
    datasets=_QWEN25_DATASETS,
    metrics=[
        ("exact_match_fmt", "EM (%)"),
        ("llm_judge_accuracy_fmt", "LLM (%)"),
    ],
    strategy_order=[
        "baseline", "self_consistency",
        "beam_search", "adaptive",
    ],
)
display(style_results_table(tbl))


Qwen2.5-Math-7B-Instruct: 72 aggregated configs


In [53]:
# ── Qwen2.5-Math-7B-Instruct — Core Metric Only ─────────────────────────────
# EM (%) for all math datasets (single column per dataset, no LLM distraction)

_model_df = agg_df[agg_df["model"] == "qwen25_math_7b_instruct"].copy()
_bs_exclude = (
    (_model_df["strategy"] == "beam_search")
    & (_model_df["aggregation"] == "mean")
    & (_model_df["scoring_window"].isna() | _model_df["scoring_window"].astype(str).isin(["all", "nan", "None"]))
)
_model_df = _model_df[~_bs_exclude].copy()

tbl_core = build_unified_results_table(
    _model_df,
    datasets=["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"],
    metrics="exact_match_fmt",
    strategy_order=[
        "baseline", "self_consistency",
        "beam_search", "adaptive",
    ],
)
tbl_core.columns = [f"{ds} EM (%)" for ds in tbl_core.columns]
display(style_results_table(tbl_core))

In [54]:
# ── Qwen3-8B (Thinking Mode) ──────────────────────────────────────────────────

_QWEN3_DATASETS = ["AIME 2024", "AIME 2025", "HumanEval-Plus"]
_model_df = agg_df[agg_df["model"].isin(["vllm_thinking_qwen3_8b", "qwen3_8b"])].copy()

# Exclude beam search variants: keep only (min, window=5)
_bs_mask = _model_df["strategy"] == "beam_search"
_bs_keep = (
    (_model_df["aggregation"] == "min")
    & (_model_df["scoring_window"].astype(str).isin(["5", "5.0"]))
)
_model_df = _model_df[~_bs_mask | _bs_keep].copy()

print(f"Qwen3-8B: {len(_model_df)} aggregated configs")

tbl = build_unified_results_table(
    _model_df,
    datasets=_QWEN3_DATASETS,
    metrics=[
        ("exact_match_fmt", "EM (%)"),
        ("llm_judge_accuracy_fmt", "LLM (%)"),
    ],
    strategy_order=[
        "baseline", "extended_thinking", "self_consistency",
        "beam_search", "adaptive",
    ],
)

# HumanEval-Plus: replace EM with humaneval_score, drop LLM
_he_tbl = build_unified_results_table(
    _model_df,
    datasets=["HumanEval-Plus"],
    metrics="humaneval_score_fmt",
    strategy_order=[
        "baseline", "extended_thinking", "self_consistency",
        "beam_search", "adaptive",
    ],
)
# Drop HumanEval EM & LLM columns, add Score column
tbl = tbl.drop(columns=[("HumanEval-Plus", "EM (%)"), ("HumanEval-Plus", "LLM (%)")])
tbl[("HumanEval-Plus", "Score (%)")] = _he_tbl["HumanEval-Plus"] if not _he_tbl.empty else ""
# Reorder columns to keep HumanEval-Plus last
_col_order = [(ds, m) for ds, m in tbl.columns if ds != "HumanEval-Plus"] + \
             [("HumanEval-Plus", "Score (%)")]
tbl = tbl[[c for c in _col_order if c in tbl.columns]]
tbl.columns = pd.MultiIndex.from_tuples(tbl.columns, names=["Dataset", "Metric"])

display(style_results_table(tbl))


Qwen3-8B: 43 aggregated configs


In [55]:
# ── Qwen3-8B — Core Metric Only ──────────────────────────────────────────────
# EM (%) for AIME, Score (%) for HumanEval-Plus

_model_df = agg_df[agg_df["model"].isin(["vllm_thinking_qwen3_8b", "qwen3_8b"])].copy()
_bs_mask = _model_df["strategy"] == "beam_search"
_bs_keep = (
    (_model_df["aggregation"] == "min")
    & (_model_df["scoring_window"].astype(str).isin(["5", "5.0"]))
)
_model_df = _model_df[~_bs_mask | _bs_keep].copy()

# AIME datasets use exact_match_fmt
tbl_core = build_unified_results_table(
    _model_df,
    datasets=["AIME 2024", "AIME 2025"],
    metrics="exact_match_fmt",
    strategy_order=[
        "baseline", "extended_thinking", "self_consistency",
        "beam_search", "adaptive",
    ],
)
tbl_core.columns = [f"{ds} EM (%)" for ds in tbl_core.columns]

# HumanEval-Plus uses humaneval_score_fmt
_he_core = build_unified_results_table(
    _model_df,
    datasets=["HumanEval-Plus"],
    metrics="humaneval_score_fmt",
    strategy_order=[
        "baseline", "extended_thinking", "self_consistency",
        "beam_search", "adaptive",
    ],
)
if not _he_core.empty:
    tbl_core["HumanEval-Plus Score (%)"] = _he_core["HumanEval-Plus"]
display(style_results_table(tbl_core))

In [118]:
# ── gpt-4o-mini ───────────────────────────────────────────────────────────────

_GPT_DATASETS = ["GPQA Diamond"]
_model_df = agg_df[agg_df["model"] == "gpt4o_mini"].copy()
print(f"gpt-4o-mini: {len(_model_df)} aggregated configs")

tbl = build_unified_results_table(
    _model_df,
    datasets=_GPT_DATASETS,
    metrics=[
        ("exact_match_fmt", "EM (%)"),
        ("llm_judge_accuracy_fmt", "LLM (%)"),
    ],
    strategy_order=[
        "baseline", "self_consistency", "offline_best_of_n", "adaptive",
    ],
)
display(style_results_table(tbl))


gpt-4o-mini: 0 aggregated configs
No data found.


In [1]:
# ── LaTeX Export ──────────────────────────────────────────────────────────────

def to_latex(df: pd.DataFrame, caption: str, label: str) -> str:
    """Convert a DataFrame to a booktabs LaTeX table string."""
    latex = df.to_latex(
        index=True,
        escape=True,
        na_rep="--",
        caption=caption,
        label=label,
        position="htbp",
    )
    return latex


beam_df = agg_df[agg_df["strategy"] == "beam_search"].copy()

# Re-generate tables and export as LaTeX
latex_outputs = []

# Strategy x Scorer tables
for dataset_label in sorted(agg_df["project_label"].dropna().unique()):
    tbl = make_comparison_table(
        agg_df,
        row_field="scorer",
        col_field="strategy",
        value_field="exact_match_fmt",
        filter_dict={"project_label": dataset_label},
    )
    if not tbl.empty:
        ltx = to_latex(
            tbl,
            caption=f"Exact match accuracy (\\%) by strategy and scorer on {dataset_label}.",
            label=f"tab:strategy_scorer_{dataset_label}",
        )
        latex_outputs.append((f"Strategy x Scorer \u2014 {dataset_label}", ltx))

# Beam search aggregation x window tables
if not beam_df.empty:
    for scorer in sorted(beam_df["scorer"].dropna().unique()):
        for dataset_label in sorted(beam_df["project_label"].dropna().unique()):
            tbl = make_comparison_table(
                beam_df,
                row_field="aggregation",
                col_field="scoring_window",
                value_field="exact_match_fmt",
                filter_dict={"scorer": scorer, "project_label": dataset_label},
            )
            if not tbl.empty:
                ltx = to_latex(
                    tbl,
                    caption=f"Beam search accuracy (\\%) \u2014 scorer={scorer}, dataset={dataset_label}.",
                    label=f"tab:beam_{scorer}_{dataset_label}",
                )
                latex_outputs.append((f"Beam {scorer} \u2014 {dataset_label}", ltx))

# Print all LaTeX
for title, ltx in latex_outputs:
    print(f"% \u2500\u2500 {title} " + "\u2500" * max(1, 60 - len(title)))
    print(ltx)
    print()


NameError: name 'pd' is not defined

In [2]:
# ── Visualization ─────────────────────────────────────────────────────────────

# Bar chart: accuracy by strategy (per dataset)
plot_df = agg_df.dropna(subset=["exact_match_mean"]).copy()

if not plot_df.empty:
    fig, axes = plt.subplots(
        1, max(1, plot_df["project_label"].nunique()),
        figsize=(6 * max(1, plot_df["project_label"].nunique()), 5),
        squeeze=False,
    )
    for idx, dataset_label in enumerate(sorted(plot_df["project_label"].unique())):
        ax = axes[0, idx]
        sub = plot_df[plot_df["project_label"] == dataset_label]
        # Average across scorers/configs per strategy
        bars = sub.groupby("strategy")["exact_match_mean"].mean().sort_values()
        bars.plot.barh(ax=ax, color="steelblue")
        ax.set_xlabel("Exact Match (%)")
        ax.set_title(dataset_label)
    plt.tight_layout()
    plt.show()
else:
    print("No data for bar chart.")

NameError: name 'agg_df' is not defined

In [30]:
# !pip install seaborn

In [31]:
# # Heatmap: beam search scorer x aggregation x window

# beam_df = agg_df[agg_df["strategy"] == "beam_search"].copy()

# if sns is None:
#     print("Install seaborn for heatmap visualization: pip install seaborn")
# elif not beam_df.empty:
#     heat_df = beam_df.dropna(subset=["exact_match_mean"]).copy()
#     heat_df["config"] = heat_df["aggregation"].astype(str) + " / w=" + heat_df["scoring_window"].astype(str)

#     for dataset_label in sorted(heat_df["project_label"].dropna().unique()):
#         sub = heat_df[heat_df["project_label"] == dataset_label]
#         if sub.empty:
#             continue
#         pivot = sub.pivot_table(
#             index="config", columns="scorer",
#             values="exact_match_mean", aggfunc="first",
#         )
#         if pivot.empty:
#             continue

#         fig, ax = plt.subplots(figsize=(max(6, pivot.shape[1] * 2), max(4, pivot.shape[0] * 0.6)))
#         sns.heatmap(pivot, annot=True, fmt=".1f", cmap="YlGnBu", ax=ax)
#         ax.set_title(f"Beam Search Accuracy — {dataset_label}")
#         plt.tight_layout()
#         plt.show()
# else:
#     print("No beam search data for heatmap.")

## Offline BoN Multi-Scorer Analysis

Download `candidates.json` from multi-scorer offline BoN runs and re-analyze
with different scorers, aggregation methods, and scoring windows.

In [32]:
import sys
import json as _json
from pathlib import Path as _Path

# Add project root and scripts/ to path so we can import analyze_candidates
_project_root = _Path.cwd().parent if _Path.cwd().name == "notebooks" else _Path.cwd()
for _p in [str(_project_root), str(_project_root / "scripts")]:
    if _p not in sys.path:
        sys.path.insert(0, _p)

# Suppress ANTLR version mismatch spam (bare print() in Recognizer.checkVersion)
import antlr4

def _noop_check(*args, **kwargs):
    pass

antlr4.Recognizer.checkVersion = _noop_check

import analyze_candidates as _ac_mod
import importlib
importlib.reload(_ac_mod)

from analyze_candidates import (
    aggregate_scores,
    select_best_candidate,
    precompute_correctness,
    analyze,
    analyze_all_windows,
    _save_csv,
)

print("Imported analyze_candidates functions")

ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
Imported analyze_candidates functions


In [33]:
# ── Offline BoN Registry ──────────────────────────────────────────────────────
# Maps run lists → (data_name, answer_format, display_label) for candidates.json analysis.

OFFLINE_BON_REGISTRY = [
    (MATH500_OFFLINE_BON_RUNS, "math500", "numeric", "MATH-500"),
    (MINERVA_OFFLINE_BON_RUNS, "minerva_math", "numeric", "Minerva Math"),
    (GAOKAO_OFFLINE_BON_RUNS, "gaokao2023en", "numeric", "Gaokao 2023 EN"),
    (OLYMPIAD_OFFLINE_BON_RUNS, "olympiadbench", "numeric", "OlympiadBench"),
    (AIME_24_OFFLINE_BON_RUNS, "aime2024", "numeric", "AIME 2024"),
    (AIME_25_OFFLINE_BON_RUNS, "aime2025", "numeric", "AIME 2025"),
    (HUMANEVAL_OFFLINE_BON_RUNS, "human_eval_plus", "code", "HumanEval-Plus"),
    # (GPQA_OFFLINE_BON_RUNS, "gpqa_diamond", "char", "GPQA Diamond"),
]

print(f"Registry: {len(OFFLINE_BON_REGISTRY)} dataset(s)")
for run_list, dname, fmt, label in OFFLINE_BON_REGISTRY:
    n_runs = sum(len(e.get("runs", [])) for e in run_list if e.get("group_url"))
    print(f"  {label}: {n_runs} run(s)")

Registry: 7 dataset(s)
  MATH-500: 3 run(s)
  Minerva Math: 2 run(s)
  Gaokao 2023 EN: 3 run(s)
  OlympiadBench: 3 run(s)
  AIME 2024: 3 run(s)
  AIME 2025: 3 run(s)
  HumanEval-Plus: 3 run(s)


In [49]:
# ── Download candidates.json from wandb ───────────────────────────────────────

import tempfile
import shutil

CANDIDATES_CACHE_DIR = _Path("cache/candidates")


def download_candidates(run_id: str, entity: str, project: str,
                        data_name: str, seed: int) -> _Path:
    """Download candidates.json for a run, with disk caching."""
    cache_path = CANDIDATES_CACHE_DIR / data_name / f"seed_{seed}" / run_id
    candidates_file = cache_path / "candidates.json"

    if candidates_file.exists():
        return candidates_file

    cache_path.mkdir(parents=True, exist_ok=True)
    run = api.run(f"{entity}/{project}/{run_id}")

    # Download to a temp dir then move (atomic-ish)
    with tempfile.TemporaryDirectory() as tmp:
        try:
            f = run.file("candidates.json")
            f.download(root=tmp, replace=True)
            src = _Path(tmp) / "candidates.json"
            shutil.move(str(src), str(candidates_file))
            print(f"  Downloaded → {candidates_file}")
        except Exception as e:
            print(f"  ERROR downloading candidates.json for {run_id}: {e}")
            return None

    return candidates_file


# Download all candidates.json files
candidates_store = {}   # (data_name, seed) -> list[dict]
run_meta = {}           # (data_name, seed) -> metadata dict

for run_list, data_name, answer_format, project_label in OFFLINE_BON_REGISTRY:
    print(f"\n{project_label} ({data_name})")
    for entry in run_list:
        if not entry.get("group_url") or not entry.get("runs"):
            continue
        for run_entry in entry["runs"]:
            seed = run_entry["seed"]
            run_url = run_entry.get("run_url", "")
            if not run_url:
                continue
            info = parse_run_url(run_url)
            cpath = download_candidates(
                info["run_id"], info["entity"], info["project"],
                data_name, seed,
            )
            if cpath and cpath.exists():
                with open(cpath) as fh:
                    data = _json.load(fh)
                candidates_store[(data_name, seed)] = data
                run_meta[(data_name, seed)] = {
                    "project_label": project_label,
                    "answer_format": answer_format,
                    "data_name": data_name,
                    "run_id": info["run_id"],
                }
                print(f"  seed={seed}  samples={len(data)}")

print(f"\nLoaded {len(candidates_store)} candidate file(s)")

# Enrich run_meta with wandb summary metrics (EM, LLM judge, HumanEval score)
_enriched = 0
for (data_name, seed), meta in run_meta.items():
    rid = meta["run_id"]
    match = raw_df[raw_df["run_id"] == rid]
    if match.empty:
        continue
    r = match.iloc[0]
    meta["wandb_exact_match"] = r.get("exact_match")
    meta["wandb_llm_accuracy"] = r.get("llm_judge_accuracy")
    meta["wandb_humaneval_correct"] = r.get("humaneval_correct")
    meta["wandb_humaneval_incorrect"] = r.get("humaneval_incorrect")
    _hc, _hi = r.get("humaneval_correct"), r.get("humaneval_incorrect")
    if pd.notna(_hc) and pd.notna(_hi) and (_hc + _hi) > 0:
        meta["wandb_humaneval_score"] = _hc / (_hc + _hi)
    else:
        meta["wandb_humaneval_score"] = None
    _enriched += 1
print(f"Enriched {_enriched}/{len(run_meta)} run_meta entries with wandb metrics")


MATH-500 (math500)
  seed=42  samples=500
  seed=43  samples=500
  seed=44  samples=500

Minerva Math (minerva_math)
  seed=42  samples=272
  seed=43  samples=272

Gaokao 2023 EN (gaokao2023en)
  seed=42  samples=385
  seed=43  samples=385
  seed=44  samples=385

OlympiadBench (olympiadbench)
  seed=42  samples=675
  seed=43  samples=675
  seed=44  samples=675

AIME 2024 (aime2024)
  seed=42  samples=30
  seed=43  samples=30
  seed=44  samples=30

AIME 2025 (aime2025)
  seed=42  samples=30
  seed=43  samples=30
  seed=44  samples=30

HumanEval-Plus (human_eval_plus)
  seed=42  samples=164
  seed=43  samples=164
  seed=44  samples=164

Loaded 20 candidate file(s)
Enriched 20/20 run_meta entries with wandb metrics


In [50]:
# ── Multi-scorer analysis across all windows ──────────────────────────────────
# Computes accuracy for every (scorer × aggregation × window) combo per dataset/seed.
# Cached: skips analysis if scoring_analysis.csv already exists next to candidates.json.

import csv

bon_records = []

for (data_name, seed), candidates_data in tqdm(sorted(candidates_store.items()), desc="Analyzing"):
    meta = run_meta[(data_name, seed)]
    label = meta["project_label"]
    run_cache = CANDIDATES_CACHE_DIR / data_name / f"seed_{seed}" / meta["run_id"]
    csv_path = run_cache / "scoring_analysis.csv"

    if csv_path.exists():
        # Load cached results
        _llm_acc = meta.get("wandb_llm_accuracy")
        with open(csv_path) as f:
            reader = csv.DictReader(f)
            for row in reader:
                bon_records.append({
                    "dataset": label,
                    "data_name": data_name,
                    "seed": seed,
                    "scorer": row["scorer"],
                    "aggregation": row["aggregation"] or "—",
                    "window": row["window"] or "all",
                    "accuracy": float(row["exact_match"]) * 100,
                    "wandb_llm_accuracy": _llm_acc,
                    "wandb_humaneval_score": meta.get("wandb_humaneval_score"),
                })
        print(f"{label} seed={seed}: loaded from {csv_path}")
        continue

    # Run analysis
    all_results, correctness, oracle_acc, max_steps = analyze_all_windows(
        candidates_data, data_name, meta["answer_format"],
    )
    _llm_ref = meta.get("wandb_llm_accuracy")
    _llm_str = f", LLM(wandb)={_llm_ref*100:.1f}%" if _llm_ref and pd.notna(_llm_ref) else ""
    print(f"{label} seed={seed}: {len(candidates_data)} samples, "
          f"max_steps={max_steps}, oracle={oracle_acc*100:.1f}%{_llm_str}")

    # Save CSV next to candidates.json
    _save_csv(all_results, csv_path, oracle_acc=oracle_acc)

    for window_label, scorer_dict in all_results.items():
        wval = window_label.split("=", 1)[1]
        for scorer, agg_dict in scorer_dict.items():
            for agg_method, accuracy in agg_dict.items():
                bon_records.append({
                    "dataset": label,
                    "data_name": data_name,
                    "seed": seed,
                    "scorer": scorer,
                    "aggregation": agg_method,
                    "window": wval,
                    "accuracy": accuracy * 100,
                    "wandb_llm_accuracy": meta.get("wandb_llm_accuracy"),
                    "wandb_humaneval_score": meta.get("wandb_humaneval_score"),
                })
        bon_records.append({
            "dataset": label,
            "data_name": data_name,
            "seed": seed,
            "scorer": "oracle",
            "aggregation": "—",
            "window": wval,
            "accuracy": oracle_acc * 100,
            "wandb_llm_accuracy": meta.get("wandb_llm_accuracy"),
            "wandb_humaneval_score": meta.get("wandb_humaneval_score"),
        })

bon_analysis_df = pd.DataFrame(bon_records)
print(f"\nTotal records: {len(bon_analysis_df)}")
bon_analysis_df.sample(10)

Analyzing:   0%|          | 0/20 [00:00<?, ?it/s]

Analyzing:  10%|█         | 2/20 [00:00<00:01, 12.51it/s]

AIME 2024 seed=42: loaded from cache/candidates/aime2024/seed_42/sx8686xk/scoring_analysis.csv
AIME 2024 seed=43: loaded from cache/candidates/aime2024/seed_43/yma1zbec/scoring_analysis.csv
AIME 2024 seed=44: loaded from cache/candidates/aime2024/seed_44/jrvn9z5v/scoring_analysis.csv
AIME 2025 seed=42: loaded from cache/candidates/aime2025/seed_42/1cwevp55/scoring_analysis.csv
AIME 2025 seed=43: loaded from cache/candidates/aime2025/seed_43/ahbar3j1/scoring_analysis.csv
AIME 2025 seed=44: loaded from cache/candidates/aime2025/seed_44/rj0hw4tp/scoring_analysis.csv
Gaokao 2023 EN seed=42: loaded from cache/candidates/gaokao2023en/seed_42/qijvr95c/scoring_analysis.csv
Gaokao 2023 EN seed=43: loaded from cache/candidates/gaokao2023en/seed_43/jboxbdly/scoring_analysis.csv
Gaokao 2023 EN seed=44: loaded from cache/candidates/gaokao2023en/seed_44/nh8fx6pk/scoring_analysis.csv
HumanEval-Plus seed=42: loaded from cache/candidates/human_eval_plus/seed_42/z6vgbfcb/scoring_analysis.csv


Analyzing: 100%|██████████| 20/20 [00:00<00:00, 84.39it/s]


HumanEval-Plus seed=43: loaded from cache/candidates/human_eval_plus/seed_43/gij2g92w/scoring_analysis.csv
HumanEval-Plus seed=44: loaded from cache/candidates/human_eval_plus/seed_44/hoitoy74/scoring_analysis.csv
MATH-500 seed=42: loaded from cache/candidates/math500/seed_42/c35z6knc/scoring_analysis.csv
MATH-500 seed=43: loaded from cache/candidates/math500/seed_43/d7jh7cbj/scoring_analysis.csv
MATH-500 seed=44: loaded from cache/candidates/math500/seed_44/cz45vmb2/scoring_analysis.csv
Minerva Math seed=42: loaded from cache/candidates/minerva_math/seed_42/7syo0ks4/scoring_analysis.csv
Minerva Math seed=43: loaded from cache/candidates/minerva_math/seed_43/y6udmktu/scoring_analysis.csv
OlympiadBench seed=42: loaded from cache/candidates/olympiadbench/seed_42/scg2r5g6/scoring_analysis.csv
OlympiadBench seed=43: loaded from cache/candidates/olympiadbench/seed_43/lrbsr0b6/scoring_analysis.csv
OlympiadBench seed=44: loaded from cache/candidates/olympiadbench/seed_44/gzpylxq2/scoring_anal

,dataset,data_name,seed,scorer,aggregation,window,accuracy,wandb_llm_accuracy,wandb_humaneval_score
101778,AIME 2024,aime2024,43,prm,mean,6173,83.33,0.733333,NaN
11329,AIME 2024,aime2024,43,entropy,mean,1608,60.00,0.733333,NaN
24882,AIME 2024,aime2024,43,entropy,max,847,70.00,0.733333,NaN
76220,AIME 2024,aime2024,43,perplexity,min,2086,70.00,0.733333,NaN
117056,AIME 2024,aime2024,43,prm,max,7137,76.67,0.733333,NaN
134049,AIME 2024,aime2024,43,sequence_prob,min,2659,66.67,0.733333,NaN
84690,AIME 2024,aime2024,43,perplexity,max,3399,66.67,0.733333,NaN
16519,AIME 2024,aime2024,43,entropy,mean,6798,60.00,0.733333,NaN
147213,AIME 2024,aime2024,43,sequence_prob,product,1509,80.00,0.733333,NaN
87647,AIME 2024,aime2024,43,perplexity,max,6356,66.67,0.733333,NaN


In [51]:
# ── Aggregate offline BoN results across seeds ────────────────────────────────

bon_agg_df = (
    bon_analysis_df
    .groupby(["dataset", "scorer", "aggregation", "window"])
    .agg(
        accuracy_mean=("accuracy", "mean"),
        accuracy_std=("accuracy", "std"),
        accuracy_count=("accuracy", "count"),
        llm_accuracy_mean=("wandb_llm_accuracy", "mean"),
        humaneval_score_mean=("wandb_humaneval_score", "mean"),
    )
    .reset_index()
)

bon_agg_df["accuracy_fmt"] = bon_agg_df.apply(
    lambda r: (
        f"{r['accuracy_mean']:.1f} ± {r['accuracy_std']:.1f}"
        if pd.notna(r["accuracy_std"]) and r["accuracy_count"] > 1
        else f"{r['accuracy_mean']:.1f}"
    ),
    axis=1,
)

bon_agg_df["llm_accuracy_fmt"] = bon_agg_df.apply(
    lambda r: f"{r['llm_accuracy_mean']*100:.1f}" if pd.notna(r["llm_accuracy_mean"]) else "",
    axis=1,
)

bon_agg_df["humaneval_score_fmt"] = bon_agg_df.apply(
    lambda r: f"{r['humaneval_score_mean']*100:.1f}" if pd.notna(r["humaneval_score_mean"]) else "",
    axis=1,
)

bon_agg_df["strategy"] = "offline_best_of_n"

print(f"Aggregated: {len(bon_agg_df)} rows")
print(f"Datasets:   {sorted(bon_agg_df['dataset'].unique())}")
print(f"Scorers:    {sorted(bon_agg_df['scorer'].unique())}")
print(f"Windows:    {sorted(bon_agg_df['window'].unique(), key=lambda w: (0, int(w)) if w != 'all' else (1, 0))}")

# Sanity check: show window=all, aggregation=mean — with baseline/self-consistency for reference
# Uses same (Strategy, Scorer) MultiIndex format with (Dataset, Metric) columns

_bon_sub = bon_agg_df[
    (bon_agg_df["window"] == "all") & (bon_agg_df["aggregation"].isin(["mean", "\u2014"]))
].copy()

_SCORER_ORDER = ["oracle", "prm", "entropy", "perplexity", "sequence_prob", "pd_gap"]

_SANITY_GROUPS = {
    "Qwen2.5-Math-7B-Instruct": ["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"],
    "Qwen3-8B": ["AIME 2024", "AIME 2025", "HumanEval-Plus"],
}

# Per-dataset metric definitions: (metric_label, agg_df_col, bon_em_col, bon_ref_col)
_DS_METRICS = {
    "HumanEval-Plus": [("Score (%)", "exact_match_fmt", "accuracy_fmt", None)],
}
_DEFAULT_METRICS = [
    ("EM (%)", "exact_match_fmt", "accuracy_fmt", None),
    ("LLM (%)", "llm_judge_accuracy_fmt", "llm_accuracy_fmt", None),
]

def _get_ds_metrics(ds):
    return _DS_METRICS.get(ds, _DEFAULT_METRICS)

for _name, _datasets in _SANITY_GROUPS.items():
    rows = []
    row_index = []

    col_tuples = []
    for ds in _datasets:
        for ml, *_ in _get_ds_metrics(ds):
            col_tuples.append((ds, ml))

    # Baseline row
    _bl = agg_df[agg_df["strategy"] == "baseline"]
    _bl_row = {}
    for ds in _datasets:
        m = _bl[_bl["project_label"] == ds]
        for ml, agg_col, _, _ in _get_ds_metrics(ds):
            _bl_row[(ds, ml)] = m.iloc[0].get(agg_col, "") if not m.empty else ""
    rows.append(_bl_row)
    row_index.append(("Baseline", "\u2014"))

    # Self-consistency row
    _sc = agg_df[agg_df["strategy"] == "self_consistency"]
    _sc_row = {}
    for ds in _datasets:
        m = _sc[_sc["project_label"] == ds]
        for ml, agg_col, _, _ in _get_ds_metrics(ds):
            _sc_row[(ds, ml)] = m.iloc[0].get(agg_col, "") if not m.empty else ""
    rows.append(_sc_row)
    row_index.append(("Self-Consistency", "\u2014"))

    # Offline BoN per-scorer rows
    for scorer in _SCORER_ORDER:
        _sr = _bon_sub[_bon_sub["scorer"] == scorer]
        row = {}
        for ds in _datasets:
            m = _sr[_sr["dataset"] == ds]
            for ml, _, bon_col, bon_ref_col in _get_ds_metrics(ds):
                if ml == "LLM (%)":
                    row[(ds, ml)] = ""
                elif not m.empty:
                    row[(ds, ml)] = m.iloc[0].get(bon_ref_col or bon_col, "")
                else:
                    row[(ds, ml)] = ""
        rows.append(row)
        row_index.append(("Offline BoN", scorer))

    mi_rows = pd.MultiIndex.from_tuples(row_index, names=["Strategy", "Scorer"])
    mi_cols = pd.MultiIndex.from_tuples(col_tuples, names=["Dataset", "Metric"])
    tbl = pd.DataFrame(rows, index=mi_rows, columns=mi_cols)
    print(f"\n{_name} (window=all, aggregation=mean)")
    display(style_results_table(tbl))

Aggregated: 165047 rows
Datasets:   ['AIME 2024', 'AIME 2025', 'Gaokao 2023 EN', 'HumanEval-Plus', 'MATH-500', 'Minerva Math', 'OlympiadBench']
Scorers:    ['entropy', 'oracle', 'pd_gap', 'perplexity', 'prm', 'sequence_prob']
Windows:    ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '


Qwen3-8B (window=all, aggregation=mean)


In [52]:
# ── Best scoring window per (scorer, aggregation) — averaged across datasets & seeds ──
# For each model group, find the window that maximizes accuracy per (scorer, aggregation).
# Window is chosen by averaging the core metric across datasets.

# Core metric per dataset (used for averaging when selecting best window/aggregation)
CORE_METRIC = {
    "MATH-500": "exact_match_fmt",
    "OlympiadBench": "exact_match_fmt",
    "Minerva Math": "exact_match_fmt",
    "Gaokao 2023 EN": "exact_match_fmt",
    "AIME 2024": "exact_match_fmt",
    "AIME 2025": "exact_match_fmt",
    "HumanEval-Plus": "exact_match_fmt",  # filled from correct/incorrect
}

CORE_METRIC_LABEL = {
    "MATH-500": "EM (%)",
    "OlympiadBench": "EM (%)",
    "Minerva Math": "EM (%)",
    "Gaokao 2023 EN": "EM (%)",
    "AIME 2024": "EM (%)",
    "AIME 2025": "EM (%)",
    "HumanEval-Plus": "Score (%)",
}

MODEL_GROUPS = {
    "Qwen2.5-Math-7B-Instruct": ["MATH-500", "OlympiadBench", "Gaokao 2023 EN"],  # Minerva Math excluded (anomalous results)
    "Qwen3-8B": ["AIME 2024", "AIME 2025", "HumanEval-Plus"],
}

AGGREGATIONS = ["mean", "min", "max", "product"]
CANDIDATE_WINDOWS = ["all", "1", "2", "3", "5", "10", "15", "20", "25", "50"]

best_windows = {}  # (model_group, scorer, aggregation) -> {window, accuracy}

for model_name, datasets in MODEL_GROUPS.items():

    # Filter bon_agg_df to this model's datasets, exclude oracle
    mask = (
        bon_agg_df["dataset"].isin(datasets)
        & (bon_agg_df["scorer"] != "oracle")
    )
    sub = bon_agg_df[mask].copy()

    if sub.empty:
        continue

    # Only consider windows that exist for ALL datasets in the group
    _window_sets = []
    for ds in datasets:
        ds_windows = set(sub[sub["dataset"] == ds]["window"].unique())
        if ds_windows:
            _window_sets.append(ds_windows)
    common_windows = set.intersection(*_window_sets) if _window_sets else set()
    common_windows = common_windows & set(CANDIDATE_WINDOWS)
    sub = sub[sub["window"].isin(common_windows)].copy()

    # For each (scorer, aggregation, window): average accuracy_mean across datasets
    # Note: bon_agg_df accuracy is always EM-based (from candidates.json re-analysis)
    window_perf = (
        sub.groupby(["scorer", "aggregation", "window"])["accuracy_mean"]
        .mean()
        .reset_index()
        .rename(columns={"accuracy_mean": "avg_accuracy"})
    )

    # For each (scorer, aggregation): find the best window
    for scorer in sorted(sub["scorer"].unique()):
        for agg in AGGREGATIONS:
            sa_df = window_perf[
                (window_perf["scorer"] == scorer) & (window_perf["aggregation"] == agg)
            ]
            if sa_df.empty:
                continue
            best = sa_df.loc[sa_df["avg_accuracy"].idxmax()]
            best_windows[(model_name, scorer, agg)] = {
                "window": best["window"],
                "accuracy": best["avg_accuracy"],
            }


# ── Display tables: one per (model_group, aggregation) ───────────────────────
_SCORER_ORDER = ["entropy", "perplexity", "sequence_prob", "pd_gap", "prm"]

# Per-dataset metric definitions
_DS_METRICS = {
    "HumanEval-Plus": [("Score (%)", "exact_match_fmt", "accuracy_fmt", None)],
}
_DEFAULT_METRICS = [
    ("EM (%)", "exact_match_fmt", "accuracy_fmt", None),
    ("LLM (%)", "llm_judge_accuracy_fmt", "llm_accuracy_fmt", None),
]

def _get_ds_metrics(ds):
    return _DS_METRICS.get(ds, _DEFAULT_METRICS)

for model_name, datasets in MODEL_GROUPS.items():
    for agg in AGGREGATIONS:
        rows = []
        row_index = []

        col_tuples = []
        for ds in datasets:
            for ml, *_ in _get_ds_metrics(ds):
                col_tuples.append((ds, ml))

        # Baseline row
        _bl_row = {}
        for ds in datasets:
            _bl = agg_df[(agg_df["strategy"] == "baseline") & (agg_df["project_label"] == ds)]
            for ml, agg_col, _, _ in _get_ds_metrics(ds):
                _bl_row[(ds, ml)] = _bl.iloc[0].get(agg_col, "") if not _bl.empty else ""
        rows.append(_bl_row)
        row_index.append(("Baseline", "\u2014"))

        # Self-consistency row
        _sc_row = {}
        for ds in datasets:
            _sc = agg_df[(agg_df["strategy"] == "self_consistency") & (agg_df["project_label"] == ds)]
            for ml, agg_col, _, _ in _get_ds_metrics(ds):
                _sc_row[(ds, ml)] = _sc.iloc[0].get(agg_col, "") if not _sc.empty else ""
        rows.append(_sc_row)
        row_index.append(("Self-Consistency", "\u2014"))

        # Oracle row
        _orc_row = {}
        for ds in datasets:
            _orc = bon_agg_df[
                (bon_agg_df["dataset"] == ds)
                & (bon_agg_df["scorer"] == "oracle")
                & (bon_agg_df["window"] == "all")
            ]
            for ml, _, bon_col, bon_ref_col in _get_ds_metrics(ds):
                if ml == "LLM (%)":
                    _orc_row[(ds, ml)] = ""
                elif not _orc.empty:
                    _orc_row[(ds, ml)] = _orc.iloc[0].get(bon_ref_col or bon_col, "")
                else:
                    _orc_row[(ds, ml)] = ""
        rows.append(_orc_row)
        row_index.append(("Oracle", "\u2014"))

        # Offline BoN per-scorer rows (best window per scorer)
        for scorer in _SCORER_ORDER:
            bw = best_windows.get((model_name, scorer, agg))
            if bw is None:
                continue
            w = bw["window"]
            row = {}
            for ds in datasets:
                match = bon_agg_df[
                    (bon_agg_df["dataset"] == ds)
                    & (bon_agg_df["scorer"] == scorer)
                    & (bon_agg_df["aggregation"] == agg)
                    & (bon_agg_df["window"] == w)
                ]
                for ml, _, bon_col, bon_ref_col in _get_ds_metrics(ds):
                    if ml == "LLM (%)":
                        row[(ds, ml)] = ""  # LLM judge not available for offline re-scoring
                    elif not match.empty:
                        row[(ds, ml)] = match.iloc[0].get(bon_ref_col or bon_col, "")
                    else:
                        row[(ds, ml)] = ""
            rows.append(row)
            row_index.append((f"Offline BoN\n(best window, {agg})", f"{scorer} (w={w})"))

        if not rows:
            continue

        mi_rows = pd.MultiIndex.from_tuples(row_index, names=["Strategy", "Scorer"])
        mi_cols = pd.MultiIndex.from_tuples(col_tuples, names=["Dataset", "Metric"])
        tbl = pd.DataFrame(rows, index=mi_rows, columns=mi_cols)

        print(f"\n{model_name} — aggregation={agg}")
        print(f"  Best window selected per scorer by averaging EM across: {', '.join(datasets)}")
        if model_name == "Qwen2.5-Math-7B-Instruct":
            print(f"  Note: Minerva Math excluded from averaging (anomalous results)")
        print(f"  Core metric per dataset: {', '.join(f'{ds}={CORE_METRIC_LABEL[ds]}' for ds in datasets)}")
        display(style_results_table(tbl))


Qwen2.5-Math-7B-Instruct — aggregation=mean
  Best window selected per scorer by averaging EM across: MATH-500, OlympiadBench, Gaokao 2023 EN
  Note: Minerva Math excluded from averaging (anomalous results)
  Core metric per dataset: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)



Qwen2.5-Math-7B-Instruct — aggregation=min
  Best window selected per scorer by averaging EM across: MATH-500, OlympiadBench, Gaokao 2023 EN
  Note: Minerva Math excluded from averaging (anomalous results)
  Core metric per dataset: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)



Qwen2.5-Math-7B-Instruct — aggregation=max
  Best window selected per scorer by averaging EM across: MATH-500, OlympiadBench, Gaokao 2023 EN
  Note: Minerva Math excluded from averaging (anomalous results)
  Core metric per dataset: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)



Qwen2.5-Math-7B-Instruct — aggregation=product
  Best window selected per scorer by averaging EM across: MATH-500, OlympiadBench, Gaokao 2023 EN
  Note: Minerva Math excluded from averaging (anomalous results)
  Core metric per dataset: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)



Qwen3-8B — aggregation=mean
  Best window selected per scorer by averaging EM across: AIME 2024, AIME 2025, HumanEval-Plus
  Core metric per dataset: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)



Qwen3-8B — aggregation=min
  Best window selected per scorer by averaging EM across: AIME 2024, AIME 2025, HumanEval-Plus
  Core metric per dataset: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)



Qwen3-8B — aggregation=max
  Best window selected per scorer by averaging EM across: AIME 2024, AIME 2025, HumanEval-Plus
  Core metric per dataset: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)



Qwen3-8B — aggregation=product
  Best window selected per scorer by averaging EM across: AIME 2024, AIME 2025, HumanEval-Plus
  Core metric per dataset: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)


In [114]:
# ── Best aggregation per scorer — using core metric ──────────────────────────
# For each (model_group, scorer), find the aggregation that maximizes the core metric.
# Uses the best window already selected in the previous cell.

best_aggregations = {}  # (model_group, scorer) -> {aggregation, window, accuracy}

for model_name, datasets in MODEL_GROUPS.items():
    print(f"\n{model_name}")
    print(f"  Core metrics: {', '.join(f'{ds}={CORE_METRIC_LABEL[ds]}' for ds in datasets)}")

    for scorer in sorted(set(sc for (mg, sc, _) in best_windows if mg == model_name)):
        best_acc = -1
        best_agg = None
        best_win = None

        for agg in AGGREGATIONS:
            bw = best_windows.get((model_name, scorer, agg))
            if bw is None:
                continue
            # Use the best window for this (scorer, agg), compute core-metric average
            w = bw["window"]
            core_vals = []
            for ds in datasets:
                match = bon_agg_df[
                    (bon_agg_df["dataset"] == ds)
                    & (bon_agg_df["scorer"] == scorer)
                    & (bon_agg_df["aggregation"] == agg)
                    & (bon_agg_df["window"] == w)
                ]
                if not match.empty:
                    core_vals.append(match.iloc[0]["accuracy_mean"])
            if core_vals:
                avg = sum(core_vals) / len(core_vals)
                if avg > best_acc:
                    best_acc = avg
                    best_agg = agg
                    best_win = w

        if best_agg:
            best_aggregations[(model_name, scorer)] = {
                "aggregation": best_agg,
                "window": best_win,
                "accuracy": best_acc,
            }
            print(f"  {scorer:<16s}  best: {best_agg} (w={best_win}, avg={best_acc:.1f}%)")

# ── Display: best (aggregation, window) per scorer ───────────────────────────
_SCORER_ORDER = ["entropy", "perplexity", "sequence_prob", "pd_gap", "prm"]

_DS_METRICS = {
    "HumanEval-Plus": [("Score (%)", "exact_match_fmt", "accuracy_fmt", None)],
}
_DEFAULT_METRICS = [
    ("EM (%)", "exact_match_fmt", "accuracy_fmt", None),
    ("LLM (%)", "llm_judge_accuracy_fmt", "llm_accuracy_fmt", None),
]

def _get_ds_metrics(ds):
    return _DS_METRICS.get(ds, _DEFAULT_METRICS)

for model_name, datasets in MODEL_GROUPS.items():
    rows = []
    row_index = []

    col_tuples = []
    for ds in datasets:
        for ml, *_ in _get_ds_metrics(ds):
            col_tuples.append((ds, ml))

    # Baseline
    _bl_row = {}
    for ds in datasets:
        _bl = agg_df[(agg_df["strategy"] == "baseline") & (agg_df["project_label"] == ds)]
        for ml, agg_col, _, _ in _get_ds_metrics(ds):
            _bl_row[(ds, ml)] = _bl.iloc[0].get(agg_col, "") if not _bl.empty else ""
    rows.append(_bl_row)
    row_index.append(("Baseline", "\u2014"))

    # Self-consistency
    _sc_row = {}
    for ds in datasets:
        _sc = agg_df[(agg_df["strategy"] == "self_consistency") & (agg_df["project_label"] == ds)]
        for ml, agg_col, _, _ in _get_ds_metrics(ds):
            _sc_row[(ds, ml)] = _sc.iloc[0].get(agg_col, "") if not _sc.empty else ""
    rows.append(_sc_row)
    row_index.append(("Self-Consistency", "\u2014"))

    # Oracle
    _orc_row = {}
    for ds in datasets:
        _orc = bon_agg_df[
            (bon_agg_df["dataset"] == ds)
            & (bon_agg_df["scorer"] == "oracle")
            & (bon_agg_df["window"] == "all")
        ]
        for ml, _, bon_col, bon_ref_col in _get_ds_metrics(ds):
            if ml == "LLM (%)":
                _orc_row[(ds, ml)] = ""
            elif not _orc.empty:
                _orc_row[(ds, ml)] = _orc.iloc[0].get(bon_ref_col or bon_col, "")
            else:
                _orc_row[(ds, ml)] = ""
    rows.append(_orc_row)
    row_index.append(("Oracle", "\u2014"))

    # Offline BoN per-scorer with best (aggregation, window)
    for scorer in _SCORER_ORDER:
        ba = best_aggregations.get((model_name, scorer))
        if ba is None:
            continue
        agg, w = ba["aggregation"], ba["window"]
        row = {}
        for ds in datasets:
            match = bon_agg_df[
                (bon_agg_df["dataset"] == ds)
                & (bon_agg_df["scorer"] == scorer)
                & (bon_agg_df["aggregation"] == agg)
                & (bon_agg_df["window"] == w)
            ]
            for ml, _, bon_col, bon_ref_col in _get_ds_metrics(ds):
                if ml == "LLM (%)":
                    row[(ds, ml)] = ""
                elif not match.empty:
                    row[(ds, ml)] = match.iloc[0].get(bon_ref_col or bon_col, "")
                else:
                    row[(ds, ml)] = ""
        rows.append(row)
        row_index.append(("Offline BoN\n(best agg+window)", f"{scorer} ({agg}, w={w})"))

    mi_rows = pd.MultiIndex.from_tuples(row_index, names=["Strategy", "Scorer"])
    mi_cols = pd.MultiIndex.from_tuples(col_tuples, names=["Dataset", "Metric"])
    tbl = pd.DataFrame(rows, index=mi_rows, columns=mi_cols)

    print(f"\n{model_name} — best aggregation & window per scorer")
    print(f"  Core metric per dataset: {', '.join(f'{ds}={CORE_METRIC_LABEL[ds]}' for ds in datasets)}")
    display(style_results_table(tbl))


Qwen2.5-Math-7B-Instruct
  Core metrics: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)
  entropy           best: product (w=5, avg=65.6%)
  pd_gap            best: mean (w=5, avg=66.8%)
  perplexity        best: mean (w=5, avg=67.1%)
  prm               best: product (w=all, avg=69.2%)
  sequence_prob     best: product (w=5, avg=66.0%)

Qwen3-8B
  Core metrics: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)
  entropy           best: max (w=all, avg=75.0%)
  pd_gap            best: max (w=all, avg=75.0%)
  perplexity        best: max (w=all, avg=74.0%)
  prm               best: min (w=15, avg=78.8%)
  sequence_prob     best: mean (w=1, avg=76.8%)

Qwen2.5-Math-7B-Instruct — best aggregation & window per scorer
  Core metric per dataset: MATH-500=EM (%), OlympiadBench=EM (%), Gaokao 2023 EN=EM (%)



Qwen3-8B — best aggregation & window per scorer
  Core metric per dataset: AIME 2024=EM (%), AIME 2025=EM (%), HumanEval-Plus=Score (%)


## Model-Specific Results Tables

Combined tables showing baseline, self-consistency, offline BoN (per-scorer from
`candidates.json` analysis), and MUR (from wandb summary) for each model.

In [115]:
# ── Model results table builder ───────────────────────────────────────────────

SCORER_ORDER = ["prm", "entropy", "perplexity", "sequence_prob"]

STRATEGY_DISPLAY = {
    "baseline": "Baseline",
    "extended_thinking": "Ext. Thinking",
    "self_consistency": "Self-Consistency",
    "offline_bon": "Offline BoN",
    "adaptive_scaling": "MUR",
}

STRATEGY_ORDER = ["baseline", "extended_thinking", "self_consistency",
                  "offline_bon", "adaptive_scaling"]


def _fmt_metric(val, std=None, count=None):
    """Format a metric value, optionally with ± std."""
    if pd.isna(val):
        return ""
    if std is not None and pd.notna(std) and count is not None and count > 1:
        return f"{val:.1f} ± {std:.1f}"
    return f"{val:.1f}"


def _get_wandb_row(model_df, strategy, scorer, dataset):
    """Extract metric values from the wandb agg_df for one (strategy, scorer, dataset)."""
    mask = model_df["strategy"] == strategy
    mask &= model_df["project_label"] == dataset
    if scorer is not None:
        mask &= model_df["scorer"] == scorer
    sub = model_df[mask]
    if sub.empty:
        return "", "", ""
    r = sub.iloc[0]
    em = r.get("exact_match_fmt", _fmt_metric(r.get("exact_match_mean")))
    llm = _fmt_metric(r.get("llm_judge_accuracy_mean"),
                      r.get("llm_judge_accuracy_std"),
                      r.get("llm_judge_accuracy_count"))
    tf = _fmt_metric(r.get("total_tflops_mean"),
                     r.get("total_tflops_std"),
                     r.get("total_tflops_count"))
    return em, llm, tf


def build_model_results_table(
    agg_df,
    bon_agg_df,
    model_filter,
    datasets,
    bon_window="all",
    bon_aggregation="mean",
):
    """Build a MultiIndex DataFrame for one model's results.

    Parameters
    ----------
    agg_df : pd.DataFrame
        Seed-averaged wandb summary data.
    bon_agg_df : pd.DataFrame
        Seed-averaged offline BoN multi-scorer analysis data.
    model_filter : str or list[str]
        model short name(s) to include from agg_df.
    datasets : list[str]
        Ordered list of project_label names to show as columns.
    bon_window : str
        Scoring window for offline BoN rows ("all", "3", etc.).
    bon_aggregation : str
        Aggregation method for offline BoN rows ("mean", "min", etc.).

    Returns
    -------
    pd.DataFrame with MultiIndex rows (Strategy, Scorer) and cols (Dataset, Metric).
    """
    if isinstance(model_filter, str):
        model_filter = [model_filter]

    model_df = agg_df[agg_df["model"].isin(model_filter)].copy()

    rows = []
    row_index = []
    metrics = ["EM (%)", "LLM (%)", "TFlops"]

    for strat_key in STRATEGY_ORDER:
        strat_label = STRATEGY_DISPLAY[strat_key]

        if strat_key == "offline_bon":
            # Get TFlops from wandb (same for all scorers)
            tflops_by_ds = {}
            for ds in datasets:
                _, _, tf = _get_wandb_row(model_df, strat_key, None, ds)
                tflops_by_ds[ds] = tf

            for scorer in SCORER_ORDER:
                row = {}
                for ds in datasets:
                    # EM from offline BoN analysis
                    mask = (
                        (bon_agg_df["dataset"] == ds)
                        & (bon_agg_df["scorer"] == scorer)
                        & (bon_agg_df["aggregation"] == bon_aggregation)
                        & (bon_agg_df["window"] == str(bon_window))
                    )
                    match = bon_agg_df[mask]
                    if not match.empty:
                        row[(ds, "EM (%)")] = match.iloc[0]["accuracy_fmt"]
                    else:
                        row[(ds, "EM (%)")] = ""
                    row[(ds, "LLM (%)")] = ""  # not available per-scorer
                    row[(ds, "TFlops")] = tflops_by_ds.get(ds, "")
                rows.append(row)
                row_index.append((strat_label, scorer))

        elif strat_key == "adaptive_scaling":
            for scorer in SCORER_ORDER:
                row = {}
                for ds in datasets:
                    em, llm, tf = _get_wandb_row(model_df, strat_key, scorer, ds)
                    row[(ds, "EM (%)")] = em
                    row[(ds, "LLM (%)")] = llm
                    row[(ds, "TFlops")] = tf
                rows.append(row)
                row_index.append((strat_label, scorer))

        else:
            # Single-row strategies (baseline, ext_thinking, self_consistency)
            strat_data = model_df[model_df["strategy"] == strat_key]
            if strat_data.empty:
                continue  # skip if no data for this strategy
            row = {}
            for ds in datasets:
                em, llm, tf = _get_wandb_row(model_df, strat_key, None, ds)
                row[(ds, "EM (%)")] = em
                row[(ds, "LLM (%)")] = llm
                row[(ds, "TFlops")] = tf
            rows.append(row)
            row_index.append((strat_label, "—"))

    if not rows:
        print("No data found for this model / dataset combination.")
        return pd.DataFrame()

    mi_rows = pd.MultiIndex.from_tuples(row_index, names=["Strategy", "Scorer"])
    mi_cols = pd.MultiIndex.from_tuples(
        [(ds, m) for ds in datasets for m in metrics],
        names=["Dataset", "Metric"],
    )

    result = pd.DataFrame(rows, index=mi_rows, columns=mi_cols)
    return result



def model_table_to_latex(df, caption, label):
    """Convert model results DataFrame to LaTeX with \\multirow strategy grouping.

    Produces a booktabs table with:
    - \\multirow for strategies spanning multiple scorer rows
    - \\multicolumn for dataset headers spanning metric sub-columns
    - \\cmidrule separators between strategy groups
    """
    if df.empty:
        return ""

    datasets = df.columns.get_level_values(0).unique().tolist()
    metrics = df.columns.get_level_values(1).unique().tolist()
    n_metrics = len(metrics)

    # Group rows by strategy
    strategy_groups = []
    current_strat = None
    current_rows = []
    for (strat, scorer) in df.index:
        if strat != current_strat:
            if current_rows:
                strategy_groups.append((current_strat, current_rows))
            current_strat = strat
            current_rows = []
        current_rows.append((scorer, df.loc[(strat, scorer)]))
    if current_rows:
        strategy_groups.append((current_strat, current_rows))

    # Column spec: l l | rrr | rrr | ...
    inner = "r" * n_metrics
    col_spec = "ll" + "".join([inner for _ in datasets])

    lines = []
    lines.append("\\begin{table}[htbp]")
    lines.append("\\centering")
    lines.append(f"\\caption{{{caption}}}")
    lines.append(f"\\label{{{label}}}")
    lines.append(f"\\begin{{tabular}}{{{col_spec}}}")
    lines.append("\\toprule")

    # Header row 1: Strategy, Scorer, dataset names spanning metrics
    h1 = ["\\textbf{Strategy}", "\\textbf{Scorer}"]
    for ds in datasets:
        h1.append(f"\\multicolumn{{{n_metrics}}}{{c}}{{\\textbf{{{ds}}}}}")
    lines.append(" & ".join(h1) + " \\\\")

    # Cmidrules under each dataset
    cmidrules = []
    col_start = 3
    for i in range(len(datasets)):
        s = col_start + i * n_metrics
        e = s + n_metrics - 1
        cmidrules.append(f"\\cmidrule(lr){{{s}-{e}}}")
    lines.append(" ".join(cmidrules))

    # Header row 2: metric names
    h2 = ["", ""]
    for _ in datasets:
        for m in metrics:
            h2.append(m)
    lines.append(" & ".join(h2) + " \\\\")
    lines.append("\\midrule")

    # Data rows
    for g_idx, (strat, scorer_rows) in enumerate(strategy_groups):
        n_rows = len(scorer_rows)
        for i, (scorer, row_data) in enumerate(scorer_rows):
            parts = []
            if i == 0 and n_rows > 1:
                parts.append(f"\\multirow{{{n_rows}}}{{*}}{{{strat}}}")
            elif i == 0:
                parts.append(strat)
            else:
                parts.append("")

            parts.append(scorer if scorer != "\u2014" else "\\textemdash")

            for ds in datasets:
                for m in metrics:
                    val = row_data.get((ds, m), "")
                    cell = str(val).strip() if pd.notna(val) and str(val).strip() else "--"
                    parts.append(cell)

            lines.append(" & ".join(parts) + " \\\\")

        # Add midrule between strategy groups (not after the last one)
        if g_idx < len(strategy_groups) - 1:
            lines.append("\\midrule")

    lines.append("\\bottomrule")
    lines.append("\\end{tabular}")
    lines.append("\\end{table}")

    return "\n".join(lines)

print("build_model_results_table() & model_table_to_latex() defined")

build_model_results_table() & model_table_to_latex() defined


In [45]:
# ── Qwen2.5-Math-7B-Instruct (Non-Thinking Mode) ─────────────────────────────

QWEN25_DATASETS = ["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"]
QWEN25_MODEL = "qwen25_math_7b_instruct"
REPORT_WINDOWS = ["all", "3", "5"]

for w in REPORT_WINDOWS:
    print(f"\n{'='*80}")
    print(f"Qwen2.5-Math-7B-Instruct — scoring window = {w}")
    print(f"{'='*80}")
    tbl = build_model_results_table(
        agg_df, bon_agg_df,
        model_filter=QWEN25_MODEL,
        datasets=QWEN25_DATASETS,
        bon_window=w,
    )
    if not tbl.empty:
        display(tbl)


Qwen2.5-Math-7B-Instruct — scoring window = all


Dataset                             MATH-500                             OlympiadBench                               Minerva Math                              Gaokao 2023 EN              \
Metric                                EM (%)     LLM (%)          TFlops        EM (%)     LLM (%)           TFlops        EM (%)     LLM (%)           TFlops         EM (%)     LLM (%)   
Strategy         Scorer                                                                                                                                                                     
Baseline         —              83.2 +/- 0.0  83.9 ± 0.1    5133.2 ± 0.0  39.3 +/- 0.0  43.1 ± 0.1     9683.7 ± 0.0  42.3 +/- 0.0  47.7 ± 0.2     3062.2 ± 0.0   68.6 +/- 0.0  73.2 ± 0.1   
Self-Consistency —              86.4 +/- 0.5  86.9 ± 0.5  35680.2 ± 69.6  44.7 +/- 0.6  46.8 ± 0.9  69723.3 ± 464.0  44.5 +/- 0.7  51.1 ± 0.7  20792.9 ± 127.1   72.4 +/- 1.2  75.2 ± 0.9   
Offline BoN      prm              87.6 ± 0.7                                45.2 ± 1.7                                 40.3 ± 9.1                                  74.2 ± 0.8               
                 entropy          85.1 ± 1.0                                40.3 ± 0.9                                 36.0 ± 7.8                                  69.8 ± 1.1               
                 perplexity       86.0 ± 0.5                                42.0 ± 0.9                                 38.8 ± 8.6                                  71.4 ± 0.8               
                 sequence_prob    85.9 ± 0.2                                40.7 ± 1.1                                 39.7 ± 7.8                                  71.0 ± 0.4               
MUR              prm                                                                                                                                                                        
                 entropy                                                                                                                                                                    
                 perplexity                                                                                                                                                                 
                 sequence_prob                                                                                                                                                              

Dataset                                         
Metric                                  TFlops  
Strategy         Scorer                         
Baseline         —                4119.7 ± 0.0  
Self-Consistency —              28817.1 ± 85.1  
Offline BoN      prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob                  
MUR              prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob


Qwen2.5-Math-7B-Instruct — scoring window = 3


Dataset                             MATH-500                             OlympiadBench                               Minerva Math                              Gaokao 2023 EN              \
Metric                                EM (%)     LLM (%)          TFlops        EM (%)     LLM (%)           TFlops        EM (%)     LLM (%)           TFlops         EM (%)     LLM (%)   
Strategy         Scorer                                                                                                                                                                     
Baseline         —              83.2 +/- 0.0  83.9 ± 0.1    5133.2 ± 0.0  39.3 +/- 0.0  43.1 ± 0.1     9683.7 ± 0.0  42.3 +/- 0.0  47.7 ± 0.2     3062.2 ± 0.0   68.6 +/- 0.0  73.2 ± 0.1   
Self-Consistency —              86.4 +/- 0.5  86.9 ± 0.5  35680.2 ± 69.6  44.7 +/- 0.6  46.8 ± 0.9  69723.3 ± 464.0  44.5 +/- 0.7  51.1 ± 0.7  20792.9 ± 127.1   72.4 +/- 1.2  75.2 ± 0.9   
Offline BoN      prm              87.2 ± 0.2                                44.6 ± 1.1                                 39.7 ± 8.8                                  74.0 ± 0.3               
                 entropy          84.5 ± 1.0                                40.5 ± 0.2                                 35.3 ± 7.3                                  69.4 ± 1.4               
                 perplexity       86.0 ± 0.3                                42.5 ± 0.6                                 38.0 ± 7.5                                  71.1 ± 0.4               
                 sequence_prob    85.9 ± 0.1                                41.0 ± 0.5                                 39.3 ± 6.2                                  70.3 ± 0.8               
MUR              prm                                                                                                                                                                        
                 entropy                                                                                                                                                                    
                 perplexity                                                                                                                                                                 
                 sequence_prob                                                                                                                                                              

Dataset                                         
Metric                                  TFlops  
Strategy         Scorer                         
Baseline         —                4119.7 ± 0.0  
Self-Consistency —              28817.1 ± 85.1  
Offline BoN      prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob                  
MUR              prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob


Qwen2.5-Math-7B-Instruct — scoring window = 5


Dataset                             MATH-500                             OlympiadBench                               Minerva Math                              Gaokao 2023 EN              \
Metric                                EM (%)     LLM (%)          TFlops        EM (%)     LLM (%)           TFlops        EM (%)     LLM (%)           TFlops         EM (%)     LLM (%)   
Strategy         Scorer                                                                                                                                                                     
Baseline         —              83.2 +/- 0.0  83.9 ± 0.1    5133.2 ± 0.0  39.3 +/- 0.0  43.1 ± 0.1     9683.7 ± 0.0  42.3 +/- 0.0  47.7 ± 0.2     3062.2 ± 0.0   68.6 +/- 0.0  73.2 ± 0.1   
Self-Consistency —              86.4 +/- 0.5  86.9 ± 0.5  35680.2 ± 69.6  44.7 +/- 0.6  46.8 ± 0.9  69723.3 ± 464.0  44.5 +/- 0.7  51.1 ± 0.7  20792.9 ± 127.1   72.4 +/- 1.2  75.2 ± 0.9   
Offline BoN      prm              87.2 ± 0.4                                45.0 ± 1.5                                 39.7 ± 8.8                                  74.2 ± 0.8               
                 entropy          85.1 ± 1.1                                40.9 ± 0.9                                 36.2 ± 7.5                                  70.7 ± 1.1               
                 perplexity       86.0 ± 0.5                                42.4 ± 0.2                                 38.6 ± 8.3                                  72.7 ± 0.7               
                 sequence_prob    85.3 ± 0.6                                40.8 ± 0.6                                 39.0 ± 6.8                                  71.6 ± 1.3               
MUR              prm                                                                                                                                                                        
                 entropy                                                                                                                                                                    
                 perplexity                                                                                                                                                                 
                 sequence_prob                                                                                                                                                              

Dataset                                         
Metric                                  TFlops  
Strategy         Scorer                         
Baseline         —                4119.7 ± 0.0  
Self-Consistency —              28817.1 ± 85.1  
Offline BoN      prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob                  
MUR              prm                            
                 entropy                        
                 perplexity                     
                 sequence_prob

In [46]:
# ── Qwen3-8B (Thinking Mode) ──────────────────────────────────────────────────

QWEN3_DATASETS = ["AIME 2024", "AIME 2025"]
QWEN3_MODEL = ["vllm_thinking_qwen3_8b", "qwen3_8b"]
REPORT_WINDOWS_QWEN3 = ["all"]

for w in REPORT_WINDOWS_QWEN3:
    print(f"\n{'='*80}")
    print(f"Qwen3-8B — scoring window = {w}")
    print(f"{'='*80}")
    tbl = build_model_results_table(
        agg_df, bon_agg_df,
        model_filter=QWEN3_MODEL,
        datasets=QWEN3_DATASETS,
        bon_window=w,
    )
    if not tbl.empty:
        display(tbl)


Qwen3-8B — scoring window = all


Dataset                            AIME 2024                                   AIME 2025                              
Metric                                EM (%)     LLM (%)            TFlops        EM (%)     LLM (%)            TFlops
Strategy         Scorer                                                                                               
Baseline         —              75.6 +/- 2.7  75.6 ± 2.7    6626.9 ± 153.1  64.4 +/- 6.2  64.4 ± 6.2    8083.6 ± 305.2
Ext. Thinking    —              78.9 +/- 5.1  78.9 ± 5.1  26477.7 ± 1594.4  66.7 +/- 3.3  66.7 ± 3.3  27573.9 ± 1140.5
Self-Consistency —              82.2 +/- 1.9  82.2 ± 1.9   52797.2 ± 816.3  34.4 +/- 6.9  73.3 ± 5.8  53684.9 ± 1499.8
Offline BoN      prm              78.9 ± 1.9                                  67.8 ± 5.1                              
                 entropy          66.7 ± 3.3                                  58.9 ± 7.7                              
                 perplexity       64.4 ± 5.1                                  60.0 ± 3.3                              
                 sequence_prob    68.9 ± 9.6                                 64.4 ± 10.7                              
MUR              prm                                                                                                  
                 entropy                                                                                              
                 perplexity                                                                                           
                 sequence_prob

In [47]:
# ── gpt-4o-mini ───────────────────────────────────────────────────────────────

GPT4OMINI_DATASETS = ["GPQA Diamond"]
GPT4OMINI_MODEL = "gpt4o_mini"
REPORT_WINDOWS_GPT = ["all"]

for w in REPORT_WINDOWS_GPT:
    print(f"\n{'='*80}")
    print(f"gpt-4o-mini — scoring window = {w}")
    print(f"{'='*80}")
    tbl = build_model_results_table(
        agg_df, bon_agg_df,
        model_filter=GPT4OMINI_MODEL,
        datasets=GPT4OMINI_DATASETS,
        bon_window=w,
    )
    if not tbl.empty:
        display(tbl)


gpt-4o-mini — scoring window = all


Dataset                   GPQA Diamond               
Metric                          EM (%) LLM (%) TFlops
Strategy    Scorer                                   
Offline BoN prm                                      
            entropy                                  
            perplexity                               
            sequence_prob                            
MUR         prm                                      
            entropy                                  
            perplexity                               
            sequence_prob

In [48]:
# ── LaTeX Export — Model-Specific Tables ──────────────────────────────────────

MODEL_CONFIGS = [
    ("Qwen2.5-Math-7B-Instruct", "qwen25_math_7b_instruct",
     ["MATH-500", "OlympiadBench", "Minerva Math", "Gaokao 2023 EN"],
     ["all"]),
    ("Qwen3-8B", ["vllm_thinking_qwen3_8b", "qwen3_8b"],
     ["AIME 2024", "AIME 2025"],
     ["all"]),
    ("gpt-4o-mini", "gpt4o_mini",
     ["GPQA Diamond"],
     ["all"]),
]

model_latex = []

for model_name, model_filter, datasets, windows in MODEL_CONFIGS:
    for w in windows:
        tbl = build_model_results_table(
            agg_df, bon_agg_df,
            model_filter=model_filter,
            datasets=datasets,
            bon_window=w,
        )
        if tbl.empty:
            continue
        safe_name = model_name.lower().replace(" ", "_").replace("-", "_").replace(".", "")
        ltx = model_table_to_latex(
            tbl,
            caption=(
                f"Results for {model_name} (scoring window={w}). "
                "EM and LLM columns show accuracy (\\%), TFlops shows total compute."
            ),
            label=f"tab:{safe_name}_w{w}",
        )
        model_latex.append((f"{model_name} (window={w})", ltx))

for title, ltx in model_latex:
    print(f"% ── {title} " + "─" * max(1, 60 - len(title)))
    print(ltx)
    print()


% ── Qwen2.5-Math-7B-Instruct (window=all) ───────────────────────
\begin{table}[htbp]
\caption{Results for Qwen2.5-Math-7B-Instruct (scoring window=all). EM and LLM columns show accuracy (\%), TFlops shows total compute.}
\label{tab:qwen25_math_7b_instruct_wall}
\begin{tabular}{llllllllllllll}
\toprule
 & Dataset & \multicolumn{3}{r}{MATH-500} & \multicolumn{3}{r}{OlympiadBench} & \multicolumn{3}{r}{Minerva Math} & \multicolumn{3}{r}{Gaokao 2023 EN} \\
 & Metric & EM (\%) & LLM (\%) & TFlops & EM (\%) & LLM (\%) & TFlops & EM (\%) & LLM (\%) & TFlops & EM (\%) & LLM (\%) & TFlops \\
Strategy & Scorer &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
Baseline & — & 83.2 +/- 0.0 & 83.9 ± 0.1 & 5133.2 ± 0.0 & 39.3 +/- 0.0 & 43.1 ± 0.1 & 9683.7 ± 0.0 & 42.3 +/- 0.0 & 47.7 ± 0.2 & 3062.2 ± 0.0 & 68.6 +/- 0.0 & 73.2 ± 0.1 & 4119.7 ± 0.0 \\
\cline{1-14}
Self-Consistency & — & 86.4 +/- 0.5 & 86.9 ± 0.5 & 35680.2 ± 69.6 & 44.7 +/- 0.6 & 46.8 ± 0.9 & 69723.3 ± 464.0 & 44.5 +/- 0.7 & 51.1 ± 0.7 & 